*Copyright (c) Cornac Authors. All rights reserved.*

*Licensed under the Apache 2.0 License.*

# Text to Graph Transformation

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PreferredAI/cornac/blob/master/tutorials/text_to_graph.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PreferredAI/cornac/blob/master/tutorials/text_to_graph.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Assume that we are working with some recommender model leveraging relationships among items, i.e., item network. However, for some dataset of interest, the latter modality is not available. Instead, we have access to another type of auxiliary information, let's say item textual descriptions. Rather than ignoring the available auxiliary information or changing to another algorithm integrating item texts, which could be costly, Cornac enables us to exploit the available textual information while using our model. This is possible thanks to Cornac's cross-modality transformation feature, which we will rely on in this tutorial.

Instructions on how to install Cornac are available [here](https://github.com/PreferredAI/cornac#installation). From now on we assume that installation is completed.

## Data loading

Consider the Amazon Clothing dataset consisting of user-item ratings and item content information (e.g., text, visual features, context). For the purpose of this tutorial, assume that the item textual descriptions is our only source of auxiliary information. This dataset is accessible through Cornac, and we can simply load it as follows:

In [6]:
!pip install cornac

In [7]:
import cornac
from cornac.data import Reader
from cornac.datasets import movielens

plots, movie_ids = movielens.load_plot()

# movies without plots are filtered out by `cornac.data.Reader`
ml_100k = movielens.load_feedback(reader=Reader(item_set=movie_ids))

In [8]:
len(ml_100k)

96297

## From Text to Graph

Recall that we are interested in a recommender model leveraging item network. We therefore seek to transform our text auxiliary information into a graph one, i.e., build an item graph encoding textual similarities between them. As we shall see below, Cornac makes this exercise convenient and straightforward thanks to its *Modality* support.

We first need to build a vector-based representation of our raw texts, i.e., item-word matrix. Importantly, we don't have to worry about how to generate such representation, the Cornac's `TextModality` class implements the necessary routines for this purpose:

In [9]:
from cornac.data import TextModality
from cornac.data.text import BaseTokenizer

# build text modality
item_text_modality = TextModality(corpus=plots, ids=movie_ids,
                                tokenizer=BaseTokenizer(sep=' ', stop_words='english'),
                                max_vocab=5000, max_doc_freq=0.5)
item_text_modality.build()

# get the item-word count matrix
item_word_mat = item_text_modality.count_matrix.A

In [10]:
item_text_modality.corpus

['A\tboy\ts\tfavorite\ttoy\tWoody\tis\tthreatened\tby\tthe\tarrival\tof\ta\tnew\ttoy\tBuzz\tLightyear\tJealous\tWoody\ttries\tto\tget\trid\tof\tBuzz\tbut\tboth\tend\tup\tlost\tThey\tmust\tfind\ta\tway\tback\tto\ttheir\towner\tbefore\the\tmoves\taway\tbut\tthey\tmust\tfirst\tface\ta\tdangerous\ttoy\tkiller\t|',
 'Agent\tJames\tBond\t007\tmust\tretrieve\tstolen\taccess\tcodes\tfor\tGoldenEye\ta\tspace\tweapon\tcapable\tof\tdestroying\tEarth\tfrom\ta\tmastermind\tand\this\tseductive\tassassin\tWith\this\tlicense\tto\tkill\tBond\traces\tagainst\ttime\tto\tsave\tthe\tworld\tfrom\tcertain\tdisaster\t|',
 'On\tNew\tYear\ts\tEve\tat\tthe\tMon\tSignor\tHotel\ta\tbellhop\tnamed\tTed\texperiences\ta\tseries\tof\tcomedic\tmishaps\tHe\thelps\ta\tcoven\tof\twitches\tdelivers\tice\tto\tthe\twrong\troom\twatches\ta\tgangster\ts\tkids\tand\treferees\ta\twager\tThe\tmovie\tis\ta\tslapstick\tcomedy\tthat\thighlights\tphysical\tcomedy\tand\tsight\tgags\t|',
 'Chili\tPalmer\ta\tdebt\tcollector\tis\tdrawn\t

In [11]:
item_word_mat.shape

(1683, 1752)

Next, all we need to do is to instantiate a `GraphModality` from our text feature matrix as in the code below, which under the hood will construct a k-nearest neighbor graph of items, encoding textual similarities among them.

In [12]:
from cornac.data import GraphModality

item_graph_modality = GraphModality.from_feature(features=item_word_mat,ids=movie_ids,
                                                 k=5, symmetric=True)

Building KNN graph: 100%|██████████| 1683/1683 [00:01<00:00, 1241.53it/s]

Symmetrizing the graph


For more details on instantiating a `GraphModality` from features, one can refer to Cornac's [documentation](https://cornac.readthedocs.io/en/latest/data.html#module-cornac.data.graph). At this level we can proceed as if we were given some item network and fit any recommender model leveraging such auxiliary information.

## Fiting and evaluating our recommender model

As a recommendation algorithm integrating item network we use [Matrix Co-Factorization (MCF)](http://papers.www2017.com.au.s3-website-ap-southeast-2.amazonaws.com/proceedings/p1113.pdf), other choices are possible, e.g., [C2PF](https://www.ijcai.org/proceedings/2018/0370.pdf). We further include [Probabilistic Matrix Factorization (PMF)](https://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf) as a baseline to assess the impact of the auxiliary information. Recall that, without the auxiliary information MCF reduces to PMF. To measure performance, we retain three metrics, namely Precision@50, Recall@50 and RMSE. All models and metrics are already implemented in Cornac, making such an experiment straightforward.  

In [18]:
from cornac.eval_methods import RatioSplit

# train/test data spliting
ratio_split_mcf = RatioSplit(data=ml_100k, test_size=0.2, exclude_unknowns=True,
                         item_graph=item_graph_modality, verbose=True, seed=123)

# train/test data spliting
ratio_split = RatioSplit(data=ml_100k, test_size=0.1, val_size=0.1, seed=123, verbose=False)

# instantiate the recommender models
pmf = cornac.models.PMF(k=40, max_iter=100, learning_rate=0.001, verbose=True, seed=123)
mcf = cornac.models.MCF(k=40, max_iter=100, learning_rate=0.001, verbose=True, seed=123)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1557
Number of ratings = 77037
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1557
Number of ratings = 19235
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1557


In [19]:
print(ratio_split.train_set.item_graph.matrix.shape)
print(ratio_split.train_set.item_data)

(1557, 1557)
defaultdict(None, {0: ([0, 88, 407, 503, 609, 544, 483, 89, 3, 135, 825, 224, 70, 51, 112, 235, 325, 587, 332, 389, 83, 461, 46, 532, 355, 290, 160, 717, 148, 331, 590, 654, 752, 814, 132, 582, 8, 551, 597, 286, 100, 27, 696, 789, 240, 257, 878, 390, 599, 205, 20, 444, 605, 157, 413, 288, 891, 558, 333, 695, 539, 666, 105, 162, 124, 566, 110, 69, 212, 182, 1, 486, 361, 250, 360, 172, 631, 400, 219, 538, 71, 475, 123, 763, 228, 295, 193, 829, 828, 468, 523, 720, 159, 12, 348, 95, 267, 452, 481, 419, 684, 856, 368, 232, 765, 16, 509, 278, 342, 533, 190, 457, 338, 62, 780, 563, 164, 402, 612, 686, 646, 411, 559, 236, 418, 75, 340, 391, 107, 158, 604, 495, 276, 443, 678, 165, 395, 189, 65, 311, 52, 608, 246, 421, 80, 784, 639, 38, 170, 175, 868, 394, 437, 369, 93, 713, 181, 357, 133, 184, 329, 151, 732, 310, 648, 547, 292, 11, 624, 385, 358, 59, 463, 562, 40, 378, 173, 473, 174, 512, 34, 284, 101, 567, 415, 790, 518, 519, 183, 681, 270, 273, 830, 502, 35, 449, 82, 488, 136, 14

In [20]:
# NDCG = cornac.metrics.NDCG(k=7)
# MAE = cornac.metrics.MAE()
# MSE = cornac.metrics.MSE()
# RMSE = cornac.metrics.RMSE()
# FMeasure = cornac.metrics.FMeasure(k=7)
# map_e = cornac.metrics.MAP()
# auc = cornac.metrics.AUC()
# rec_20 = cornac.metrics.Recall(k=20)

NDCG = cornac.metrics.NDCG()
NDCG_3 = cornac.metrics.NDCG(k=3)
NDCG_4 = cornac.metrics.NDCG(k=4)
NDCG_5 = cornac.metrics.NDCG(k=5)
NDCG_6 = cornac.metrics.NDCG(k=6)
NDCG_7 = cornac.metrics.NDCG(k=7)
NDCG_8 = cornac.metrics.NDCG(k=8)
NDCG_9 = cornac.metrics.NDCG(k=9)
NDCG_10 = cornac.metrics.NDCG(k=10)
NDCG_11 = cornac.metrics.NDCG(k=11)
NDCG_12 = cornac.metrics.NDCG(k=12)
NDCG_13 = cornac.metrics.NDCG(k=13)
NDCG_14 = cornac.metrics.NDCG(k=14)
NDCG_15 = cornac.metrics.NDCG(k=15)
NDCG_16 = cornac.metrics.NDCG(k=16)
NDCG_17 = cornac.metrics.NDCG(k=17)
NDCG_18 = cornac.metrics.NDCG(k=18)
NDCG_19 = cornac.metrics.NDCG(k=19)
NDCG_20 = cornac.metrics.NDCG(k=20)

MRR = cornac.metrics.MRR()
MRR_3 = cornac.metrics.MRR(k=3)
MRR_4 = cornac.metrics.MRR(k=4)
MRR_5 = cornac.metrics.MRR(k=5)
MRR_6 = cornac.metrics.MRR(k=6)
MRR_7 = cornac.metrics.MRR(k=7)
MRR_8 = cornac.metrics.MRR(k=8)
MRR_9 = cornac.metrics.MRR(k=9)
MRR_10 = cornac.metrics.MRR(k=10)
MRR_11 = cornac.metrics.MRR(k=11)
MRR_12 = cornac.metrics.MRR(k=12)
MRR_13 = cornac.metrics.MRR(k=13)
MRR_14 = cornac.metrics.MRR(k=14)
MRR_15 = cornac.metrics.MRR(k=15)
MRR_16 = cornac.metrics.MRR(k=16)
MRR_17 = cornac.metrics.MRR(k=17)
MRR_18 = cornac.metrics.MRR(k=18)
MRR_19 = cornac.metrics.MRR(k=19)
MRR_20 = cornac.metrics.MRR(k=20)

MAE = cornac.metrics.MAE()
MAE_3 = cornac.metrics.MAE(k=3)
MAE_4 = cornac.metrics.MAE(k=4)
MAE_5 = cornac.metrics.MAE(k=5)
MAE_6 = cornac.metrics.MAE(k=6)
MAE_7 = cornac.metrics.MAE(k=7)
MAE_8 = cornac.metrics.MAE(k=8)
MAE_9 = cornac.metrics.MAE(k=9)
MAE_10 = cornac.metrics.MAE(k=10)
MAE_11 = cornac.metrics.MAE(k=11)
MAE_12 = cornac.metrics.MAE(k=12)
MAE_13 = cornac.metrics.MAE(k=13)
MAE_14 = cornac.metrics.MAE(k=14)
MAE_15 = cornac.metrics.MAE(k=15)
MAE_16 = cornac.metrics.MAE(k=16)
MAE_17 = cornac.metrics.MAE(k=17)
MAE_18 = cornac.metrics.MAE(k=18)
MAE_19 = cornac.metrics.MAE(k=19)
MAE_20 = cornac.metrics.MAE(k=20)

MSE = cornac.metrics.MSE()
MSE_3 = cornac.metrics.MSE(k=3)
MSE_4 = cornac.metrics.MSE(k=4)
MSE_5 = cornac.metrics.MSE(k=5)
MSE_6 = cornac.metrics.MSE(k=6)
MSE_7 = cornac.metrics.MSE(k=7)
MSE_8 = cornac.metrics.MSE(k=8)
MSE_9 = cornac.metrics.MSE(k=9)
MSE_10 = cornac.metrics.MSE(k=10)
MSE_11 = cornac.metrics.MSE(k=11)
MSE_12 = cornac.metrics.MSE(k=12)
MSE_13 = cornac.metrics.MSE(k=13)
MSE_14 = cornac.metrics.MSE(k=14)
MSE_15 = cornac.metrics.MSE(k=15)
MSE_16 = cornac.metrics.MSE(k=16)
MSE_17 = cornac.metrics.MSE(k=17)
MSE_18 = cornac.metrics.MSE(k=18)
MSE_19 = cornac.metrics.MSE(k=19)
MSE_20 = cornac.metrics.MSE(k=20)

RMSE = cornac.metrics.RMSE()
RMSE_3 = cornac.metrics.RMSE(k=3)
RMSE_4 = cornac.metrics.RMSE(k=4)
RMSE_5 = cornac.metrics.RMSE(k=5)
RMSE_6 = cornac.metrics.RMSE(k=6)
RMSE_7 = cornac.metrics.RMSE(k=7)
RMSE_8 = cornac.metrics.RMSE(k=8)
RMSE_9 = cornac.metrics.RMSE(k=9)
RMSE_10 = cornac.metrics.RMSE(k=10)
RMSE_11 = cornac.metrics.RMSE(k=11)
RMSE_12 = cornac.metrics.RMSE(k=12)
RMSE_13 = cornac.metrics.RMSE(k=13)
RMSE_14 = cornac.metrics.RMSE(k=14)
RMSE_15 = cornac.metrics.RMSE(k=15)
RMSE_16 = cornac.metrics.RMSE(k=16)
RMSE_17 = cornac.metrics.RMSE(k=17)
RMSE_18 = cornac.metrics.RMSE(k=18)
RMSE_19 = cornac.metrics.RMSE(k=19)
RMSE_20 = cornac.metrics.RMSE(k=20)

R2 = cornac.metrics.R2()
R2_3 = cornac.metrics.R2(k=3)
R2_4 = cornac.metrics.R2(k=4)
R2_5 = cornac.metrics.R2(k=5)
R2_6 = cornac.metrics.R2(k=6)
R2_7 = cornac.metrics.R2(k=7)
R2_8 = cornac.metrics.R2(k=8)
R2_9 = cornac.metrics.R2(k=9)
R2_10 = cornac.metrics.R2(k=10)
R2_11 = cornac.metrics.R2(k=11)
R2_12 = cornac.metrics.R2(k=12)
R2_13 = cornac.metrics.R2(k=13)
R2_14 = cornac.metrics.R2(k=14)
R2_15 = cornac.metrics.R2(k=15)
R2_16 = cornac.metrics.R2(k=16)
R2_17 = cornac.metrics.R2(k=17)
R2_18 = cornac.metrics.R2(k=18)
R2_19 = cornac.metrics.R2(k=19)
R2_20 = cornac.metrics.R2(k=20)

FCP = cornac.metrics.FCP()
FCP_3 = cornac.metrics.FCP(k=3)
FCP_4 = cornac.metrics.FCP(k=4)
FCP_5 = cornac.metrics.FCP(k=5)
FCP_6 = cornac.metrics.FCP(k=6)
FCP_7 = cornac.metrics.FCP(k=7)
FCP_8 = cornac.metrics.FCP(k=8)
FCP_9 = cornac.metrics.FCP(k=9)
FCP_10 = cornac.metrics.FCP(k=10)
FCP_11 = cornac.metrics.FCP(k=11)
FCP_12 = cornac.metrics.FCP(k=12)
FCP_13 = cornac.metrics.FCP(k=13)
FCP_14 = cornac.metrics.FCP(k=14)
FCP_15 = cornac.metrics.FCP(k=15)
FCP_16 = cornac.metrics.FCP(k=16)
FCP_17 = cornac.metrics.FCP(k=17)
FCP_18 = cornac.metrics.FCP(k=18)
FCP_19 = cornac.metrics.FCP(k=19)
FCP_20 = cornac.metrics.FCP(k=20)
# FMeasure = cornac.metrics.FMeasure()
# FMeasure = cornac.metrics.FMeasure(k=7)
# map_e = cornac.metrics.MAP()
# auc = cornac.metrics.AUC()
# rec_20 = cornac.metrics.Recall(k=20)


In [21]:
# instantiate and run your experiment
exp = cornac.Experiment(eval_method=ratio_split,
                        # models=[pmf,mcf],
                        models=[pmf],
                        metrics=[NDCG, NDCG_3, NDCG_4, NDCG_5, NDCG_6, NDCG_7, NDCG_8, NDCG_9, NDCG_10, NDCG_11, NDCG_12, NDCG_13, NDCG_14, NDCG_15, NDCG_16, NDCG_17, NDCG_18, NDCG_19, NDCG_20, MAE,MAE_3, MAE_4, MAE_5, MAE_6, MAE_7, MAE_8, MAE_9, MAE_10, MAE_11, MAE_12, MAE_13, MAE_14, MAE_15, MAE_16, MAE_17, MAE_18, MAE_19, MAE_20, MSE, MSE_3, MSE_4, MSE_5, MSE_6, MSE_7, MSE_8, MSE_9, MSE_10, MSE_11, MSE_12, MSE_13, MSE_14, MSE_15, MSE_16, MSE_17, MSE_18, MSE_19, MSE_20, RMSE, RMSE_3, RMSE_4, RMSE_5, RMSE_6, RMSE_7, RMSE_8, RMSE_9, RMSE_10, RMSE_11, RMSE_12, RMSE_13, RMSE_14, RMSE_15, RMSE_16, RMSE_17, RMSE_18, RMSE_19, RMSE_20, R2, R2_3, R2_4, R2_5, R2_6, R2_7, R2_8, R2_9, R2_10, R2_11, R2_12, R2_13, R2_14, R2_15, R2_16, R2_17, R2_18, R2_19, R2_20, MRR, MRR_3, MRR_4, MRR_5, MRR_6, MRR_7, MRR_8, MRR_9, MRR_10, MRR_11, MRR_12, MRR_13, MRR_14, MRR_15, MRR_16, MRR_17, MRR_18, MRR_19, MRR_20, FCP, FCP_3, FCP_4, FCP_5, FCP_6, FCP_7, FCP_8, FCP_9, FCP_10, FCP_11, FCP_12, FCP_13, FCP_14, FCP_15, FCP_16, FCP_17, FCP_18, FCP_19, FCP_20],
                        # metrics=[R2,FCP],
                        user_based=True)
exp.run()

Learning...
epoch 0, loss: 7295.620349
epoch 1, loss: 6114.710214
epoch 2, loss: 5315.881918
epoch 3, loss: 5021.160731
epoch 4, loss: 4864.523258
epoch 5, loss: 4756.641688
epoch 6, loss: 4670.321526
epoch 7, loss: 4593.858191
epoch 8, loss: 4521.888828
epoch 9, loss: 4452.393051
epoch 10, loss: 4384.995560
epoch 11, loss: 4319.809115
epoch 12, loss: 4256.830968
epoch 13, loss: 4195.824968
epoch 14, loss: 4136.460210
epoch 15, loss: 4078.465452
epoch 16, loss: 4021.687541
epoch 17, loss: 3966.035827
epoch 18, loss: 3911.395931
epoch 19, loss: 3857.590408
epoch 20, loss: 3804.390480
epoch 21, loss: 3751.587230
epoch 22, loss: 3699.026451
epoch 23, loss: 3646.621389
epoch 24, loss: 3594.349736
epoch 25, loss: 3542.243186
epoch 26, loss: 3490.373383
epoch 27, loss: 3438.836760
epoch 28, loss: 3387.739966
epoch 29, loss: 3337.188356
epoch 30, loss: 3287.278791
epoch 31, loss: 3238.095629
epoch 32, loss: 3189.709054
epoch 33, loss: 3142.175116
epoch 34, loss: 3095.537175
epoch 35, loss: 30

In [22]:
# instantiate and run your experiment
exp = cornac.Experiment(eval_method=ratio_split_mcf,
                        # models=[pmf,mcf],
                        models=[mcf],
                        metrics=[NDCG, NDCG_3, NDCG_4, NDCG_5, NDCG_6, NDCG_7, NDCG_8, NDCG_9, NDCG_10, NDCG_11, NDCG_12, NDCG_13, NDCG_14, NDCG_15, NDCG_16, NDCG_17, NDCG_18, NDCG_19, NDCG_20, MAE,MAE_3, MAE_4, MAE_5, MAE_6, MAE_7, MAE_8, MAE_9, MAE_10, MAE_11, MAE_12, MAE_13, MAE_14, MAE_15, MAE_16, MAE_17, MAE_18, MAE_19, MAE_20, MSE, MSE_3, MSE_4, MSE_5, MSE_6, MSE_7, MSE_8, MSE_9, MSE_10, MSE_11, MSE_12, MSE_13, MSE_14, MSE_15, MSE_16, MSE_17, MSE_18, MSE_19, MSE_20, RMSE, RMSE_3, RMSE_4, RMSE_5, RMSE_6, RMSE_7, RMSE_8, RMSE_9, RMSE_10, RMSE_11, RMSE_12, RMSE_13, RMSE_14, RMSE_15, RMSE_16, RMSE_17, RMSE_18, RMSE_19, RMSE_20, R2, R2_3, R2_4, R2_5, R2_6, R2_7, R2_8, R2_9, R2_10, R2_11, R2_12, R2_13, R2_14, R2_15, R2_16, R2_17, R2_18, R2_19, R2_20, MRR, MRR_3, MRR_4, MRR_5, MRR_6, MRR_7, MRR_8, MRR_9, MRR_10, MRR_11, MRR_12, MRR_13, MRR_14, MRR_15, MRR_16, MRR_17, MRR_18, MRR_19, MRR_20, FCP, FCP_3, FCP_4, FCP_5, FCP_6, FCP_7, FCP_8, FCP_9, FCP_10, FCP_11, FCP_12, FCP_13, FCP_14, FCP_15, FCP_16, FCP_17, FCP_18, FCP_19, FCP_20],
                        # metrics=[R2,FCP],
                        user_based=True)
exp.run()


[MCF] Training started!
Learning...
epoch 0, loss: 10153.945520
epoch 1, loss: 8059.104105
epoch 2, loss: 6893.547559
epoch 3, loss: 6385.309968
epoch 4, loss: 6090.033318
epoch 5, loss: 5888.913450
epoch 6, loss: 5739.780319
epoch 7, loss: 5621.891982
epoch 8, loss: 5523.157115
epoch 9, loss: 5435.570001
epoch 10, loss: 5353.386479
epoch 11, loss: 5272.607236
epoch 12, loss: 5191.161416
epoch 13, loss: 5109.041312
epoch 14, loss: 5027.917388
epoch 15, loss: 4949.953196
epoch 16, loss: 4876.738613
epoch 17, loss: 4808.892704
epoch 18, loss: 4746.227574
epoch 19, loss: 4688.065358
epoch 20, loss: 4633.503988
epoch 21, loss: 4581.613330
epoch 22, loss: 4531.565868
epoch 23, loss: 4482.707001
epoch 24, loss: 4434.593705
epoch 25, loss: 4387.011950
epoch 26, loss: 4339.958838
epoch 27, loss: 4293.590139
epoch 28, loss: 4248.128889
epoch 29, loss: 4203.772835
epoch 30, loss: 4160.626248
epoch 31, loss: 4118.671277
epoch 32, loss: 4077.780661
epoch 33, loss: 4037.755641
epoch 34, loss: 3998

Ranking: 100%|██████████| 942/942 [00:03<00:00, 311.18it/s]


TEST:
...
    | FCP@-1 | FCP@10 | FCP@11 | FCP@12 | FCP@13 | FCP@14 | FCP@15 | FCP@16 | FCP@17 | FCP@18 | FCP@19 | FCP@20 |  FCP@3 |  FCP@4 |  FCP@5 |  FCP@6 |  FCP@7 |  FCP@8 |  FCP@9 | MAE@-1 | MAE@10 | MAE@11 | MAE@12 | MAE@13 | MAE@14 | MAE@15 | MAE@16 | MAE@17 | MAE@18 | MAE@19 | MAE@20 |  MAE@3 |  MAE@4 |  MAE@5 |  MAE@6 |  MAE@7 |  MAE@8 |  MAE@9 | MSE@-1 | MSE@10 | MSE@11 | MSE@12 | MSE@13 | MSE@14 | MSE@15 | MSE@16 | MSE@17 | MSE@18 | MSE@19 | MSE@20 |  MSE@3 |  MSE@4 |  MSE@5 |  MSE@6 |  MSE@7 |  MSE@8 |  MSE@9 |   R2@-1 |   R2@10 |   R2@11 |   R2@12 |   R2@13 |   R2@14 |   R2@15 |   R2@16 |   R2@17 |   R2@18 |   R2@19 |   R2@20 |    R2@3 |    R2@4 |    R2@5 |    R2@6 |    R2@7 |    R2@8 |    R2@9 | RMSE@-1 | RMSE@10 | RMSE@11 | RMSE@12 | RMSE@13 | RMSE@14 | RMSE@15 | RMSE@16 | RMSE@17 | RMSE@18 | RMSE@19 | RMSE@20 | RMSE@3 | RMSE@4 | RMSE@5 | RMSE@6 | RMSE@7 | RMSE@8 | RMSE@9 | MRR@-1 | MRR@10 | MRR@11 | MRR@12 | MRR@13 | MRR@14 | MRR@15 | MRR@16 | MRR@17 | MRR@18 | MRR@19 

**Output:**
<pre>
    |   RMSE | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------------ + --------- + --------- + --------
PMF | 1.5082 |       0.0024 |    0.1002 |    5.6788 |   1.2293
MCF | 1.2823 |       0.0043 |    0.1846 |   10.1189 |   1.1811
</pre>

Clearly, MCF offers a substantial improvement over PMF. This highlights the importance of cross-modality transformations, which allowed us to take advantage of the available auxiliary data, namely item textual descriptions, while using the MCF model integrating item network into personalized recommendation.   

In [23]:
# import numpy as np
# arr=[1.010587883996804,0.4551658990095184,0.5069882308166864,0.5193117139840271,0.5474569221727599,0.2861893438846419,1.3642134724107375,2.763859036390082,1.9587049418486933,2.2672007846244924,0.7249423300648434,0.8629857079340228,0.7539665497795127,0.6610886334263197,0.5949865606960014,1.1627545260121765,0.4587484471218062,0.1903852548219609,0.3718916570830575,2.0932352857600645,2.9982276505034378,0.7948718447900767,1.6731381553101699,1.998631995862218,0.6173342314554873,0.9952912247488772,0.18839130374139845,0.8887749967160552,0.23149635793685963,0.12348867133746477,1.159727368083619,1.6197971012309553,1.0248107516343068,0.3207216882724291,0.1917797126207419,1.7235555524564985,1.0468445636912942,1.5563484447179121,0.2200744652783646,0.8919708225410918,0.7948566031952216,0.4232688803157494,0.5978305074075045,0.2803751946793722,0.3677086848245157,0.10036763838487373,0.5461267283499138,1.9933463742358177,0.15629595526574125,2.4781528906473866,0.2345569548221298,0.1544084887536714,0.19011998617103743,0.8342682349651316,0.36375154229475015,0.9041614335854424,0.24133063976550417,1.92382013046246,0.8314008672013644,0.9595041052289659,1.0803373582871956,0.16394673005232896,0.18101898272923478,1.2550444834880796,0.1894515614490171,0.40571046735679417,0.16115901175584924,1.568033833414448,0.17699432721378194,0.32162247650050446,0.4874409422316157,0.3289029564086299,0.9730727328753952,0.5645914418168037,3.363531037464777,0.1856805820267772,0.5984083592341127,1.3371849139307515,0.20935177741561795,0.8163502204393884,0.3045597187645459,0.25201654143999264,1.0042819365237312,0.36862148223689495,0.20036151008581626,3.0437799428738765,1.0662073452521135,0.09074160177906698,0.3538921976485492,1.1932930618110236,0.22435191617464945,0.5670435485074385,0.5951196627669404,1.0053463742678348,0.6204540082284407,1.2318513553068375,0.4784116121136044,0.5118555951566108,1.5975759625628978,0.8467696160228044,0.2504230551764124,0.39254331295566125,2.2608364569691743,2.758084568448644,1.4582191010512744,0.25274625940477513,1.7201153581442195,0.7726487773134885,1.1465738662395717,0.012836010588307634,1.1803229675392797,0.6653916194249074,0.15692420525815898,1.8029186108400341,0.9379544244239506,0.706140346714593,0.18430184421472262,1.3835631889399096,0.3540981338850493,0.31848627257045514,1.7644467992439896,1.2454072566263603,0.3993461013362216,0.3975086237565897,1.997873179050504,1.0122342428568936,0.10092510642815045,0.14381780434444913,0.3913403912514015,0.5588820865077704,0.8589431664125646,1.8863299030807985,0.25082208955331353,1.694773564859315,0.33256656319064515,0.34976993606699125,0.13620811123466892,0.17335545427872034,1.0902429970436356,0.7980846013095722,0.9950537287718157,0.7923238490868796,1.3471943336732353,0.8792274600227952,0.7245659905390168,0.46552952786973334,0.6809909763508815,0.0038957071063911997,0.3459743461550744,1.0163916805261035,0.27270826501799306,0.2842878684293506,0.5723774537194721,0.5918459714237727,0.32046062850476353,0.7030670934984506,0.11957150828354557,0.47806322477497876,0.09698499286369433,0.40361957552389693,0.2644253140354875,0.16375865926566213,0.4179348429022198,0.9810516957785742,0.8110173734186557,2.206835360272966,0.9348719558271181,0.2260701036229822,0.5618133316146272,0.4702298410562933,2.7333482148483554,0.43681183500393317,2.0047451116061,0.25881335125548155,0.6049278455765772,0.19324151078821217,0.4835150304188929,0.35223664117732145,1.7139849987120677,0.8504375978374892,0.8368248007003496,0.4037552658561907,1.3378963423634929,0.40514238987122986,0.31099930970689993,0.16624919437094565,0.14672062405731,2.1788100743812295,1.1095823233914999,0.7955439604189198,0.1262165934525763,0.5366289744731314,1.0268046412594511,2.6623126177913075,0.7083131436639194,1.6240130657875786,0.4002026647105721,0.3888253307322582,0.19815105715288706,0.21025811262928684,0.5300644531696943,0.39755514592892816,0.0205723523623611,0.6252168242189189,0.3888503235108969,0.19693648687941678,0.609696367275868,3.760356211817804,0.5168629114312107,1.7473812453136635,0.6906274019135079,0.29912446152930006,1.1652096061508281,1.00590256947471,0.27674192003291687,0.2904463584258412,0.5725465849224847,0.18600796667571906,0.5637508239658116,0.45264275759237105,0.6876940522063997,1.6667088626763071,0.21521453564742865,1.9984077003106129,0.7532878703181716,0.5342272716674543,0.6323516464402652,1.344591852802541,0.4939654052127942,0.2621082951615685,0.666604911104292,0.6456433738898522,0.5479968399393016,0.7727882129426469,0.2603531153681624,0.16781612832244797,0.02879551846073225,0.249411303542308,0.005049570649264368,0.7309471015943938,1.255212888524548,1.8679814650983602,1.2142998585690878,0.5015534259872507,1.0945719679641663,0.23266156395498494,0.003997874659703804,0.2751329946993539,1.1378846413551136,1.1736031045405575,2.0003251530655937,0.25081001791746793,0.45748467932869197,0.4198054330437424,0.4584530493476944,0.23007019507129733,1.2833915550348296,0.8800356264897191,3.3175821944059214,2.175577909516118,0.7783160340082365,0.6668742009389135,0.5905592389989689,0.16727883580350422,1.7229019644010826,1.4850631975997155,0.9622443244130907,0.7364569755894834,0.8415646269504515,0.9001755228884245,0.15221319832876823,0.47118352225175997,1.0232096631834917,0.48070164360584816,1.613797052823244,0.39884521524606065,0.6416506659993244,1.7841858081443096,0.944298042912898,0.3980581035743729,0.40542693436400096,0.9146926776868396,0.27656974066521123,1.568187295057609,0.17339240267503442,2.288226444030453,0.12628998858199658,1.453441801940519,0.8354162505050962,0.4013992065452703,0.686980468971095,1.4311174628633898,1.2274689580888358,0.44566990872623613,0.8441196628687111,1.8707924149500896,0.8610204332460842,0.6272508091345479,1.434112435327859,0.2781822114913304,0.7005074440484202,0.5972083060759559,0.867967288379389,0.580921490574003,0.6686655903827345,1.9972034222957253,0.638344923879989,0.6497830636469963,0.2116207753520385,1.2166536942922908,1.3787450595033999,0.9437744271483427,2.6522110502540786,0.7365783244414885,1.8383256916346795,0.5621095285966478,0.14404124756611125,0.7910478362822424,0.47859798364273365,0.10734865770028401,0.3896696160663007,0.5928624779587226,2.420685549266927,0.2371055371003763,0.1953113209041959,0.7336039768799107,2.004122736837987,0.8602657554582303,1.7807509462425042,1.1886680418592626,0.16446403976020196,0.9956018364187549,0.15370194835002415,0.9917636757195494,0.9847536431297463,1.8747744736966645,0.6723365229877039,0.2843406252870988,0.08176311980975726,0.13648992982410668,0.4025467058527852,0.23953159508349842,0.4879509072768067,0.2005587471016801,0.9592432441092154,1.2726141563250581,0.2799392823941975,0.9904781651296339,1.551515269691801,1.1806581332787105,2.546300129977789,0.529789212567219,0.41153466848297215,0.8693883034575858,1.5223383526711332,0.7065531723618443,1.3867816082490219,0.21669217026844212,0.6125862782902676,1.6726904453342852,0.7451454219090197,1.161035386827836,3.467207028762709,0.5620167934908888,0.2704374755525567,0.7335872284866394,1.0458782603951136,0.8804650197103312,0.24935692932598386,1.2316295044243262,0.2127463298584793,0.22006991839433,1.5978982749153907,0.4666616180694916,0.9884610056445275,0.41753034012156354,0.6243191279850002,0.16719700978969243,0.2630534255502912,0.36492520732215183,0.2411123547556837,0.468933991555879,3.3132749989110053,0.13670169936796128,0.5015928460033079,1.2339971188181522,0.21137390572736692,0.9325250499420941,0.6630103514291719,0.17389385148023173,0.9919326047384671,0.35645119256014457,1.4147449765347142,0.6942946506284002,0.15889570294900235,1.999606570112363,0.684680874117122,0.6871951572374009,0.08800559440213274,1.0402216694476363,0.6161214322455697,0.3256283079169977,0.6759193595645261,1.3956782972561563,1.9969666494434284,1.0027177510542002,1.0200051145735802,0.12868171150831564,1.4594268590498842,0.8339302552012247,2.4375596655724245,0.4437094831091457,0.24630935627898154,0.464440811233676,1.5741013076019326,0.21962804338555397,1.5471929188423932,0.004264502877290521,0.213358716266141,1.713380917342243,0.7575868410463951,0.22970512559676237,0.35185196032165944,1.4348477363043046,0.8469447301283308,1.6680415159116855,1.3154484226039522,0.7334138235285614,0.30035618094201233,0.49259290223879226,1.2067088833746893,0.2718984652116774,1.4063391688295934,0.3364905111712062,0.7684496354683361,0.6928087055323076,0.31038938955942275,1.8305325784782944,0.6796230648624197,0.36569099569652685,0.6288870625394711,0.1671887781656043,0.16512991310363212,1.7988342456444664,2.0019742916876924,1.7187770730383183,0.32675351178906276,2.0033152780255556,1.1172639101128614,0.19227449644831296,0.4580101543142874,3.5190068861118093,0.3098323680768269,0.3383471652206316,0.6873487632347661,0.7882919336993668,0.33292560655482495,1.2982982726580237,0.6980007350022923,0.7173392819990347,0.6365184657424381,2.6803118523486322,1.4852214321041064,0.15752462433065162,1.6778714223876996,0.1497271656913739,0.6100098110686272,0.417517733735572,0.1315533527623618,0.4030045032838383,1.0227328197764756,0.1645089828068098,0.12274863369431754,0.6959675447595401,1.205324480367458,0.17250687860672098,0.6333107125492106,0.2707312880789523,0.051005342653140495,0.2535631295056504,0.8906922269462672,0.32387382085600525,0.5750071257964864,1.5463288256017487,0.6345436508301798,2.15182301540934,0.16179427720426087,1.2809974081412085,1.0488955714958155,1.4537778436244402,0.40917325507272295,1.9105938495413313,0.32188675126310606,0.45212331045222065,0.4761235488373199,0.4583314377832899,0.33178782952824193,1.1720847230860647,0.945978410361608,0.9398293357830152,0.6959465434908996,0.9802648435051848,0.09548468867797766,0.7650242057688921,0.19886554440645554,0.23595752012617943,0.18639014121934538,0.14668939149837712,0.4075902361892805,0.15230186504804166,0.42096111257914526,0.995760045797605,0.12765478159612087,0.5114845803768304,0.1360388204396017,3.419955650455927,0.23385074003102613,0.5749949809236834,0.23012238358638193,0.948303996221437,2.3315253587969345,0.717921348593392,1.1233225018372068,0.7332468589038967,0.3475938861034127,0.3926574695029226,0.6451866415408117,0.44260313389770367,0.16970133549228272,1.4337106663894414,0.7105040192660301,0.1574143699926065,1.1536666196374252,0.5260773965954044,1.1992192682835068,0.7834261812013379,0.15917087295133037,1.58406425551056,0.19776988713138977,1.330732964051176,0.5288090532843697,0.3431747950160169,0.39359662262469364,0.1446308179756688,2.3807514427295313,3.2398845590402487,0.634617683005172,0.07059440300334074,2.024161146457523,0.4428762801760491,2.9104471944020505,0.7075562609282766,0.29849935174569353,1.688995634189002,0.26539674332865815,0.9981639320319049,2.0833568684201906,2.5535332463280427,1.883052659146366,1.2747989406579117,0.14396146963137824,2.0046154235974893,0.6301806136936614,0.5349776473446433,0.23676755508427805,0.1679127479929745,0.3062526432286039,0.42614057918664017,0.13838667398679405,0.4305297402173438,0.4270775508918643,1.8369803689430615,3.165690226768141,1.4691962306410407,2.5915726672122763,0.9985023145156782,1.0333071524430828,1.230964650276304,2.5238330931135495,0.5680216919586805,1.9698250026588089,0.3673091804335642,0.4907272772331764,0.2063680964663135,0.6151748777076769,0.4813020917705728,0.09146689798277219,1.2195282514686716,0.43254188858589604,0.6666623373590408,0.19066673679769064,1.737383951779199,0.8775866772195862,0.2439707152560313,0.33145367290937067,0.2140236848654684,2.4848808001816423,0.3486029376480815,0.7710763401038161,0.930056193069702,0.37136649768294916,0.5661896182530182,0.210396696262789,1.5089655110704623,0.3569809104067101,1.4798065528982027,0.7494175968115417,0.4178034124855774,1.9271570510037457,1.1878552097087143,0.26033269929313985,0.9452214575938847,2.0053921434840576,1.3942570919013124,0.6523428296353639,0.06322576091605026,0.4768381231939929,0.7251765133931194,2.003666378476777,1.6887742035688946,1.0762289519682633,0.943349530443463,3.531463553648564,1.686594924496951,1.5661790331826693,0.9508366648194766,0.43689164434142036,0.07044458806812148,1.7095940362981343,0.20706948454263951,1.8043109673868183,1.4523180933458852,0.1314940582268065,0.27306024268487716,0.1536215041734703,1.684631608291855,0.2696498136440457,2.0036147518951326,0.33320871293569887,0.5004576303967045,1.698670686918864,0.3487300623030998,1.3714286720560471,0.44023548320978545,2.2361179315587414,0.7201698817910076,1.3589275196036725,1.3657126710569323,2.1810586828352108,0.39891516788118864,0.7699345736336518,0.6375392924560335,1.2614822955424243,1.5056646790033268,0.9004236552342011,0.1853111989709877,0.3057615535599574,0.5438318749108433,2.0462164596465255,0.40558748735049477,1.4726274826362271,0.2319239069806187,0.3474302620866512,1.3640095273438688,0.19566228916970374,2.251252856505359,1.7704381585637101,0.13578116630101356,0.0014038694997853263,0.9450149460548123,2.0319671898611036,2.9606767779662992,0.3007514142189507,1.337507057588113,1.070266011046912,0.9965177534268124,1.5144759149297196,0.6617968433754893,0.32415560285327594,2.715373373353564,0.4574802562407214,0.16617913511278104,0.725106099202657,0.16523348955055805,0.12658835275264746,0.46262283020815076,1.0461109871711385,0.7157334531934634,0.21873227277786267,0.24185013794185473,0.16199655943223235,0.4032819986499625,2.135908616251387,0.20044801641781618,0.0639120596548648,1.7020695373234003,1.7289904292338156,2.005923599422308,1.7688356470047344,1.2636179563053105,0.007512453450784662,0.745136860143591,0.2990203553688193,0.13240449337283344,1.1934514787836528,0.35004974350720364,0.46616306731875135,2.1580818314616703,0.29558866138685813,0.7045943523709397,1.4735635107724177,1.3482914488813624,0.46087098994190256,1.6958406177549827,1.696305365978155,0.17461043171420343,0.18563136813021242,1.4941971498606965,0.0068053301037602765,0.3901019189314088,0.36166592237941764,1.0371559790019438,0.7824776498517139,0.3113085955298498,0.4243892204655211,0.522992112166544,0.6250353909013437,1.9282953332190207,0.40080419913443155,0.17344625822177306,0.22688648039653003,0.10461473713094449,0.24527374457612616,0.3046461505963638,0.6499802018080167,0.20841953831215676,1.4468954310320012,1.2516242567847833,0.4383882341142824,1.3967329040029832,0.5517640247921118,0.15273543537059808,1.4871790201946706,1.8673144924781142,0.6116711907464145,1.7527547661315968,0.18495650503768157,0.4992430768020135,0.8728054571476269,1.0015398793877859,0.4703105163517556,0.34964548292758657,0.5897518550446472,0.23844347879271854,0.3409799409726437,1.777738637719334,0.19268121976836206,0.7971057091740796,1.332082313028308,0.838660480155001,0.2038770465813844,0.2947203107740375,0.7944706174180229,1.1451648497899485,1.5419820242680173,1.3728227564980455,1.284625287411957,0.16039421107869245,1.4564325338312574,0.7041922983975568,0.17350500878803654,0.008620222253057808,1.1797437018935537,0.4213885341444419,0.9534105912486712,0.2555539727264229,0.3095926088043086,2.6438156390135807,0.7173296498031512,0.30118479612369065,0.41837004570463066,0.07676138589707815,1.5840149866973512,0.1646373913299346,2.415819188191219,0.3079272357798768,0.03597383354813921,0.5867780969681764,0.8790581811497677,0.9524258066894,0.13023186207961412,0.5801124275207519,0.09592534617917536,0.21666493658167774,2.003551394524753,0.0032726502043294303,0.15764028693286924,1.2370924243562018,1.7321250580535885,0.7518785441296023,1.364156429606767,0.39631625678715743,2.0039383497595344,0.9172140239216997,1.2947608728844182,0.12931010634332019,0.6838456656646343,1.178534093056399,0.4378063035144777,0.49384996596864816,0.43778392732220084,0.7065494491943252,0.10953665429687343,0.22308260114087375,0.18909681630265673,2.86886728276993,0.3320040442837371,0.22497238875392078,1.725581654976784,1.067940748883827,0.42868020668603,0.18934186378401918,0.7939838586667483,0.8391371587218988,0.002011896661453072,0.04578205981193406,0.3576972748636731,1.241326581308134,0.19985587167369268,0.5040392780265908,0.3631899091142614,0.31868133066494764,0.2562758240519418,0.6706373247625512,0.4623251832697033,1.6679099994438331,0.2038564933024709,0.748475449724423,0.6193710777527046,0.5759062701151407,0.20952043791035635,0.011935776782175278,0.2020547365326868,0.909630094394811,0.31496400661832524,0.3928229694779217,0.8986220375586775,0.7420677101268247,0.2597647056790713,0.3732927003981157,0.14689610515856621,0.40360464928511597,0.6434593937103585,0.6301951523469675,1.2555943201642634,0.1986610887355802,2.5763014646538034,0.20889030066695735,1.5058842011914253,1.0788538202349214,3.3105049932417767,0.7635635891560497,0.4005526318659296,1.2785621084331371,0.776788257018451,0.12351021726940914,0.6233577236504528,1.0154073922530764,1.3560370101764114,1.1019202947356153,0.037187793668723934,0.6877900362310179,0.8572643641231164,1.2823649781053628,0.9837185918986577,0.3049272751969805,0.004102345395506024,1.1786320232877041,2.736568820618002,0.5281026552214652,1.758951377346425,0.25611685760981917,0.5904418396110547,0.5430390091004083,1.7714353296115855,0.40441998610100516,0.838731667796802,0.8565303986990047,0.1566405180946635,0.43789920287607664,0.04649599020285944,0.6421992505770477,1.4966622026419458,0.27847466497408124,2.8611545586614677,0.4070659087686863,0.7378476227102837,0.11899272998209565,0.40113148834398515,0.21836486513150488,2.402049039437695,0.15453909911459895,0.12556273041030686,0.6506215651721039,0.5936813793720024,0.3890550631954399,0.5680392177749063,0.08558836373544088,0.6877202079036291,1.1159013898973633,1.521350373797946,0.2033140742240378,1.2264991593781949,1.8569140541577713,1.080887907606468,1.21860053888979,0.15646411611716893,0.1264699425722533,0.9297212768039707,0.19693770895747775,3.002729535730472,0.8780583861911264,0.1809942739873609,0.16857815859877334,0.16223265271397302,1.9961490644395905,2.7480949519025906,0.7461800476958498,0.7447034392061367,0.42748774920987165,1.0978821125692297,0.19531590295913226,2.0041112001094925,1.3566573981799994,1.4337074577745126,0.2833256529622288,0.5083395612845414,0.588318610597355,0.12240634526298155,0.31767633369496906,0.3641904661472246,0.5166463218517823,1.6871566159658964,0.49128153559371857,1.1976680598165341,1.8440375592345175,0.8204371838161246,0.14210994374484454,0.5556509851023588,0.3335568108424667,0.3966601013844051,0.5024441093262695,0.1406016015169138,0.9675439940705406,1.810212380882645,0.34336594508675033,0.3109296425369221,0.5841645878287514,0.027501100772120424,0.3023793743739507,0.9933789070910919,0.32177234942958677,0.9839544429132618,0.7360460695265338,1.4309010055421738,0.3100175925291664,1.0164136182915575,0.08713666850322355,0.7227048088758936,0.20649222753067775,0.9414702330319304,0.6198799524828562,0.5578896066089474,0.1522650806012642,0.6904659241700193,0.6379112088726142,0.48311400942131977,0.6389239484811622,1.0561873258584766,0.6917047905531595,1.2044539495569198,0.40609005685846356,0.5714445971482744,0.2766851258262566,0.5505724683386966,0.004943290875983664,0.2555848977539803,1.0002157000702039,3.7181318184355288,0.42907474033773685,0.03827203277222502,0.9752610331303555,2.642482085828478,0.560852750485676,2.7571775471360205,0.4538174215014732,1.19056914609016,2.52958316922669,0.14412932161203207,0.15584698406111053,2.958578596560937,0.005699747902220942,0.6221165067043586,2.1058914011499477,2.04389195190479,1.5023592796476097,2.8951154516418898,1.3812012941267247,1.4874526286399568,0.5328512497156339,0.2592541690212853,1.2060760930324474,0.3691735269979928,0.9881642166873279,0.02916512411028238,2.0343989468996746,0.9076756828095682,0.6009490243230529,0.8165231739129606,1.0552014807396244,1.449000139799669,2.1122077348010846,0.1704685158660766,0.8779270005200841,2.004875526407705,0.7337063744600041,1.2724681342094675,0.18573894815497738,0.12661008924056993,2.505424158489186,0.2606578335819627,0.17144359912927065,0.46918992425825046,1.3708224528047688,0.3277083978402753,0.16309247835015706,0.368526847942217,0.8489032585785639,0.49105853076049755,0.24380684883777626,0.6733159559033495,1.18046831971661,1.532467405139275,1.62163175489529,0.6151446310594224,2.291919061251975,0.30281550716987127,0.4288723257604783,0.2167902679576681,2.6395476391566,0.626112023834839,2.2739976211820885,0.2638926242377444,0.3882474017349553,0.5416382637820121,0.24461302879405444,0.016227299524313565,0.5041277975846707,0.6382050659477638,0.19810859485915344,1.5915922979389618,0.29170419179638696,0.11857804868350863,0.22694733300632208,0.20968485621003818,0.3344471191529905,0.761805287653929,2.788910473217123,0.555345194285108,0.2497352816437779,0.23806553450318546,0.8319568463332216,0.585869351775794,1.1111939120916352,2.0709728271834753,0.0058760267690924906,0.2120621677299237,0.3673172040137051,0.24535838573489777,0.39129500042506393,1.3561033809075695,2.450837319603642,0.7632300841696349,2.3174486069147675,0.5777493556581064,0.1996576890171804,0.2988621578368478,0.44136058092885033,1.0809860190965344,0.24521012364886818,1.3344797299748965,1.6109384509576126,2.352823864586563,1.044636716750846,0.3665844024427489,1.4757068965317748,1.8198933785884925,2.7520906670806835,0.4456888766554692,2.054477056998416,0.8986330684243287,0.05503623379761624,2.7671772034873205,2.2974352860237977,0.7970489657358373,1.7450333631856,0.11303702399819704,1.337039641968674,1.9928314550678736,0.6528314103433219,0.3781716700360418,0.21070451676044133,1.3594691498158245,0.15759266506374736,0.0023192625044585746,0.7377351085099395,0.2762674242893848,1.770904301078608,0.23183691563169795,0.46692953304362206,0.26920759019091234,0.9345990211308801,0.1897775507433881,0.1492307696634616,0.1935375769565013,2.0038973873848347,0.4915852455709757,0.17963903252549374,1.2133475695653768,1.1116780653711213,0.4110828723301174,1.9953096560383528,0.1979387566841826,1.0031687763994706,0.5807889053234749,0.44143387293852454,2.3221824561764297,0.4832350022552676,0.1848358048317813,0.5597291667635176,0.12985868476394646,0.2460369830470821,0.5868666920770593,0.13070796786221983,1.2491236888721766,1.6159628641749224,0.2756814095447142,0.1876930179594236,1.0832923234397205,0.04505443638055606,1.4779418935567925,1.5218099000215775,1.9987406318056928,0.13540149193117568,0.5368497056183701,0.35088727008300147,1.5896173283228094,1.0674378290063915,0.6912744514105214,0.16606216936402873,0.1500579029350524,1.0544694384552058,1.1244443248454155,0.31482508388017205,1.8545849709613034,2.0754945205794524,0.1574032826747631,0.5264722739310805,1.279826576924897,0.5585374149866524,0.3117895350224149,2.7244022018350407,0.19267332421324745,1.4093735476710547,1.1636505757266677,0.05593995982402333,0.8000498281859856,1.392237791889361,1.8800641130619922,1.3467652891622466,0.36265045184939115,0.3062004502862985,2.0247339496697974,0.9775930764491745,0.39048392051065983,0.9573567788505724,2.0197784113462784,0.3152647217324489,2.248435566184403,0.574145739961397,0.8500858523107464,0.0028093474638275495,0.41792853098649485,1.8047583296973877,0.155621745274253,1.6510943467408925,0.3295834608496797,0.36616150688035587,2.4551237759389264,0.7425955712478896,0.3486334778997895,0.40431815674170757,1.184137162192905,1.8340804322746678,0.9635746699657577,0.8033712316931367,0.376641211746624,1.2780075645723041,0.4660402379671025,1.1930413506093538,0.6947743330627469,0.5420117971787821,1.378466164143381,1.34885051043667,0.5389441907106196,0.14235489836192272,0.3348203299661443,0.2710247065664042,2.0044838837862846,1.0029790821328015,1.1888319977351633,0.49740580395698153,0.25612903567465395,1.1446950197020245,0.8474310429916088,1.015883382334179,0.3810996273664866,0.1867847302412846,0.21036172703372813,0.26840385299016667,1.824093634371522,2.6049114976791126,0.38825704655874205,1.1405966806927723,2.85191727601314,0.19864337508193586,1.0040044101897903,0.48876787323492277,0.425755534299082,0.3420843466043504,0.9408284570628651,0.567296775763495,1.0704268352579902,2.680351959734198,1.468016551417449,0.3731420981606348,0.14289436360357222,0.3663919730851646,1.7484374274741499,0.3122875972205419,2.805738961016506,0.23910354165642822,0.5567716482697431,0.8405081188764871,0.47374114537714096,0.5454972214722402,0.40732141306923353,0.5263224082444662,0.16628325196738647,0.22167778557320794,1.051135951172483,0.5191979571399177,0.12971497764856643,0.4257492061941317,0.47011554989149573,0.19637635232725525,1.197498337768133,0.6621458186612748,0.8683655626013769,0.2347486245721262,0.40728173186851535,2.759340753238261,0.2617140186467024,0.30382131838458903,0.1749485258509793,0.30371325494550216,0.15552636966442712,3.48444461110275,0.257304327210802,1.0423318501186545,1.2334115014862954,0.36139527420291806,0.5869378162950554,0.395958230718648,0.5528731004751073,0.8426227021499955,1.5358996539634306,0.8804187373010344,0.11643605272150914,0.13381322002193663,0.7936452106327945,0.5933983102649623,1.0025264092933206,2.573208860333417,0.267411584738495,0.18734034860898596,0.5813565801901763,0.20663709891165816,3.793194481975558,1.0474031097067442,0.3834198039531582,0.17927700243310518,0.4345491282930549,0.6888239452426781,2.893639829259664,0.21691432556806944,1.9960404247369894,1.1276752175150573,1.3429904985259897,1.547260354804671,2.6104453855535636,0.39992683651323757,1.4608384955664993,1.0475635439922755,0.6919832398742791,0.17710853716841335,2.8030461490862795,0.38764317298027784,1.0408763097204747,1.1851572644345705,0.7019560802417679,1.4683089238778835,1.1755028914797259,0.6579722381962281,0.6089916492130572,0.4357617268032481,0.3831444026853026,1.5805285304194228,0.8390128990014905,0.3364466182219825,1.6378957398602099,0.48829662223732595,1.0444438463196075,1.458424936457571,1.6640818992363622,0.3444065879296794,0.43854157257511606,1.1433109745012153,3.451029413928852,0.7424072957300822,0.7426901913698591,1.2985512631141352,0.5095088543589181,0.4815788430987116,1.0625178153829822,1.2047353016928062,3.628040563003962,0.47076118897461683,0.8040267340730676,1.0164143538827144,0.8362957664907027,0.5296576543052991,1.6682565293593568,0.790181138686425,0.844046930787381,0.5442505424366937,0.13992641900724934,0.28161267895654163,0.6591074464874926,0.14197018813371187,0.9953874720880616,1.1897049912741475,3.502455768545679,0.4349079164811753,0.9381559681349554,0.9445856576176173,0.6219236952208753,0.45829056193878426,0.7823610730762578,0.8260611005964833,0.002749536515064488,0.2658046340443585,1.2645102723057198,1.9981451039177989,0.8269664121856213,0.19673625931604022,0.4685231451462464,0.20387460274281288,0.1670630198796612,0.00753304655631748,0.32528029511376744,1.413569479466612,0.13675019834335522,0.721281218316034,1.5231060253258395,0.8668315160254103,0.676670249469776,1.4584321588222249,0.34938688909516324,1.0017318069022974,2.0672656019360205,0.2638591392548002,3.160675962751659,0.022432899379801974,0.19691022810989,2.650803813677573,0.5474104594839666,0.46821834092380765,0.4326675817557959,1.493428374849687,0.0464992110792668,2.7082611486708954,0.3925669373689846,1.6770886869136303,1.0228732449600353,0.6114610658449902,0.15961404256454692,0.7525415782633078,0.003366055901838916,1.5891329221648567,1.002966107251841,0.2309407636707128,0.19978858076516381,0.5358028746577421,1.2102868850505994,0.42878992052507137,0.30716049520999,2.3357389397688353,0.22045219895393053,0.9885841125090646,0.24651999998645202,0.16137420557004845,1.184896686599346,1.6257004498125693,2.8802900175490596,0.41148507533181267,1.6733289329646555,0.5318090595797695,1.200435328591333,0.7378616860304597,0.7995515307171148,1.1877046295628517,0.2524037144469,1.1850385621343302,1.2617400114657618,1.7781101718373673,1.0847727987952798,0.5035546686243917,0.3162678369626377,0.8758729882286459,1.5897358391259573,0.4961265973835549,0.7444062500757598,2.768014234463365,1.3525946414122088,0.37153260783442743,0.251669215708727,0.7934407792889573,0.7115255122456965,1.5479359751205224,1.4611430232931308,0.14013397185405907,0.44153718203600345,0.2442929927976194,2.2626255400830413,0.5297735173584925,0.8267242322783606,1.5987695176897276,1.9085334102838596,0.32040310501462255,0.11262015883644416,0.8595136151899645,0.40974973475701404,0.7641566574956116,0.6739653578058671,2.627362601836397,1.2721733605308936,1.7672354837259245,0.5105828557627508,1.2062948506074957,0.1829078409550835,0.16362863951964002,0.1867155807979186,0.333417639189574,1.5768812531614411,0.5608309961506581,0.7521393326068262,1.0156416282168301,0.1627453873239375,0.7202668405370529,0.7734574820343925,0.14802118648632945,0.03419650456452672,0.7304861473451574,1.1453614290500813,0.6421974476979049,0.39850297256772116,1.649983773841834,0.5736411430513275,0.29938837839356625,0.8859973061937199,3.787285979665871,0.3535877510187655,0.798854223674105,0.36494941717919405,1.7975744645990197,0.36530652195840485,0.6635509554413219,1.4741025395400893,0.2591494627523083,0.9681956155146825,0.7460523534905157,0.29457289378613716,0.8935993049773234,0.765993096130078,0.3069803270762623,2.495282514160615,2.826798767839426,0.8717355715388972,0.9062186459538033,0.13644669749813598,1.0188818051427724,0.22816113616395306,0.7203917607799029,0.5820233341619849,0.2373552396571963,0.6886164100950936,0.31953922395656953,0.9475981295989495,3.1410517329732226,1.0286913693846884,2.101875647959619,1.5589394176846545,0.8333864802520581,0.7165293272854596,0.40200431533686665,0.4427568759254399,0.85618438199708,0.2508281592312471,0.21330205533108737,0.0030971292412829143,1.9955631649133672,1.6605108405110904,0.15474682002688134,0.36661939928736054,2.9004012691960463,0.6456506367812995,0.5142329082456856,2.8864371004907623,0.5945049957091966,0.5845136324252067,1.0679467370972135,0.40696117632283,1.2971133544973714,0.702079810150564,0.6101303077407128,0.3898749854147767,0.8100419786530859,0.5954710211881373,1.0486047870728612,1.9284836719890812,0.5622337456884896,0.39802699190891033,0.6200348657756285,1.768325591341109,0.11413526865559298,0.5153740527359325,3.5250275511563984,0.7467818251829996,3.8737212836662764,0.003798577065150077,0.9654589619521747,0.33060961121562515,2.042609523170608,0.13899956228774712,1.9969765458251114,0.5187419665265072,0.2680644342830938,2.4681521740974763,1.6618897019620658,0.6661565104144316,1.3763619657550998,0.6299841670117559,1.6618416663981304,0.41645638492225423,1.2736799621905464,0.81790655546888,0.29308173206598376,1.7262575012993473,0.6589639302490167,0.0036932261324436055,0.5922163054669256,0.1396172695903779,0.42265159091402005,0.2102999382582511,0.15386737093381933,1.8707985454846665,1.1789971913546053,1.7668294024783666,1.3239109712750734,1.7316018841977012,2.379501514451815,1.0779778059343164,0.309256068793065,1.337429810689068,0.9073633755846657,0.6006376122908028,0.6922371825403264,1.1979520256432554,0.4636543164383289,0.9777636516139925,0.27211707788885775,3.133636986763801,0.02471783800595162,2.9114681520526444,0.26179945217801404,1.3806956118024223,0.2705665014391867,0.432182287350817,0.15630614760807404,0.9590328315953407,1.0918784524226255,0.8623690848716352,0.016719485820715363,0.8139164813029058,0.5641802334025279,0.041659912294449875,0.24442592581130818,0.21836084430801694,0.73849438697615,0.6808384592432546,1.530884972738145,1.341823618788863,1.231710526358122,0.2314494900782972,2.9266635499780973,0.5279207249183173,2.664663772121628,0.9873679029469623,0.8478265269271801,3.0424564044269795,0.7121325209713625,0.13434797744216365,1.5889897689643,0.4546068279437909,0.21537424646694703,2.855927052449937,1.1122153681237945,1.3875463596201527,0.25511941010537775,0.5717851613050158,0.23284881971412652,0.041642825924741445,0.25913665073543335,1.7271991701169909,0.2640520358727705,0.1888297217625432,0.7575487586804668,0.9366447000464806,0.5226815477271769,0.9872412042909386,0.47504951549963614,1.1151355089033679,0.24379209498171406,0.4017002286020519,0.7107257094539472,1.8916944003402758,1.6218544795114362,0.6654815111440757,1.057593797675115,0.23716046965314663,2.364814972372362,3.825919999416806,0.5567452702272968,2.0041756271370823,0.5328503162151996,0.5623765961282121,1.382388011126408,2.6126197775857976,0.2722649672627302,1.519473755421878,0.15923161707984002,0.6277351418495929,0.6191948663010427,0.2213722799597413,0.48543458708974185,1.4988363173670365,1.4337830628092458,0.0015685800823073137,2.0040658444412345,2.5145442577647064,0.6844174430226806,1.1812797501219086,0.18619400820286458,0.26338443326694083,0.4226415440007889,0.3720812962384006,2.0016886154616316,1.3274191841813967,0.14801741153910442,0.6879926767146136,1.3596547003171677,1.3514906342478534,2.8806596626736916,0.5440578555528663,0.33634928502047146,0.38317010171010635,0.0038678133485237076,3.4305871210561314,0.3333249752476357,0.5869918930299507,0.26768847325859646,0.3890432495038678,0.4044044662170627,0.49849057433048394,0.2637474311239636,0.37922862741931107,1.178435321343177,0.1308815904638454,0.33487720443351243,0.31159393014848913,0.5554419307769267,1.028512173964499,0.2511658424353085,0.911079163877142,0.5703991615867263,0.1980537719094233,0.84174538088556,0.6280924978384563,0.48150987446644766,0.24076283099765305,0.47506877114184753,1.175310398323854,2.7297287636424805,1.3006796898604698,0.6777301267413272,0.3193536210634438,1.4307403504763285,2.6463756360742465,0.14081619064671536,0.257495892878028,1.056255711502218,0.1933632651406656,0.20188243956584273,0.21902149008940341,2.35329250721645,0.3101282794623401,1.118015373306184,0.003996032314007358,1.6893537965122172,0.566234671709875,0.5958043366370793,1.0519001672467416,3.6087733568062674,0.6888780807938957,0.0022038152641912223,0.4845270749724717,0.7459084320127496,0.4647498210923082,2.3454541853002837,2.3114579622928457,0.5391551641113814,3.3878894224917455,1.1363667678792782,0.6230922779780599,0.44317989237089606,0.27198479755884897,1.6195682801512161,0.8469328637222517,0.5909993093304946,0.7199493357279625,0.2983632351017853,0.4711413141603549,0.3816729097901641,0.1774658774244351,0.1712667215116621,0.35087754567694773,0.8976220312361971,0.6532813691241426,0.1853945140796558,2.0475170520455324,0.3516189359385322,0.29243828335779654,0.8292794578328646,0.2668627859187036,1.203281074434909,2.9514659992408796,0.24320088785680305,3.369205098019819,3.6897190067517247,0.19602003908265253,0.6982832961091536,0.43590322722892294,0.34377196835094637,0.7498470841708142,0.4311302346926844,0.9955155092318178,1.3410093639926277,0.2427201241415311,0.4260870603871769,0.95241992181976,1.2598208673406321,0.6449617977563102,0.4927815014279293,1.3920541541712557,1.0147138827087874,0.8767817877829813,0.6330374229088616,1.4727850437270766,0.38321403898013173,1.1186647940406242,0.8055994600536958,0.8381317660973349,0.3022555293461018,0.4221548839695366,0.42051166980915866,0.40611218352893275,0.48601532074057197,3.775069440936174,2.7846936076745847,1.9954103745768048,0.4829019532055261,0.4555837172640542,0.8784425172352073,0.5026220866264914,0.5169445899912923,0.19957828205990236,0.26104190682529627,0.3018139478161119,0.39996602544267645,0.8872107561107079,0.32063532834275144,2.766865844592531,0.4705479104211019,1.356615859511764,1.2380472176710526,0.36661026763328985,1.3041862525900048,0.5007253475442521,0.5252259137239932,2.004514240660076,0.1530812127615233,1.2899958813239154,1.5024439225850381,0.2895088856653114,1.704289718372467,0.3620865113456482,0.949258524838938,0.22554403290882874,0.4706238733534107,0.17807057561663786,1.0030236696809478,0.36650947949346957,0.6198785578016746,0.32521908381198816,0.16183630352187262,0.5408862754507471,0.3529742077586411,3.2619250809758675,0.6529054118220663,0.8626203165334161,0.03540098394663804,0.09607162352431775,1.011987378137111,2.154932888762591,0.521761251631025,2.966230333836258,1.996126117528978,1.722452336613045,1.000549654463657,0.5639602361286649,0.29776200708988965,1.6216490232515044,0.0037463321607700273,0.9451296838229353,1.9813417236172306,2.805513105603386,0.5801781348791618,]
# avg=np.mean(arr)
# avg

In [24]:
# data=[[3.33069157],[5],[3.74813722],[5],[2.9270712,3.30316362],[5,5],[2.80057974],[2],[4.35533309],[5],[3.55938677],[5],[4.50478059],[2],[2.57115642],[5],[3.00238293],[2],[3.37565188],[3],[3.1534006],[5],[1.86009464,3.21997225],[2,2],[2.97493261],[2],[4.37375049],[5],[3.89635439],[4],[2.83638281,3.25126684],[4,4],[2.69337714],[5],[2.80295164],[5],[2.92611919],[5],[3.60553187],[3],[2.99688483],[5],[2.79723199],[3],[4.61157814],[5],[4.2757779],[5],[4.29152464],[5],[4.8787002],[5],[3.04322635],[5],[3.13986652],[5],[3.91454105],[4],[3.88995895,3.60802017],[5,5],[3.46887306],[3],[3.06045893],[5],[3.19347043],[3],[3.88443485],[3],[3.46841803],[5],[4.37181321,3.43207664],[5,5],[3.75003448],[5],[3.36711563],[4],[3.50903039,3.96550041,3.31052504,3.21821084],[5,5,5,5],[2.76241996],[4],[2.90084687],[2],[2.72620876],[5],[3.0019573],[5],[2.93605274],[4],[3.20407744],[2],[3.10661264],[4],[4.86325776,4.62047775],[5,5],[4.4178142],[5],[3.978227],[4],[3.73561545],[5],[3.20539362],[5],[3.30401957],[5],[3.60355885,3.66016757],[2,2],[3.27806514],[5],[4.29222094],[5],[3.88853372],[5],[3.30874041],[5],[3.1208076],[3],[4.78173367],[5],[4.07428994],[5],[4.35579882],[5],[4.92525944],[5],[3.16413171],[4],[2.98946023],[5],[4.62742423],[5],[3.30270431],[5],[2.99901094],[5],[3.26828009],[5],[4.78087479,4.82797768,4.82696168],[5,5,5],[2.46863981],[4],[4.737773,3.24410191],[5,5],[3.93836775],[5],[3.00585738],[5],[3.38006242],[5],[2.28203396],[5],[3.05607076],[4],[3.74945694],[5],[2.62999413],[5],[4.64833314],[5],[3.37960968],[5],[3.24423832],[5],[3.31442427],[5],[3.39493833],[3],[3.45325482],[5],[3.31587881],[4],[2.75113808],[5],[3.49844519],[5],[3.40380482],[4],[3.99047277],[3],[3.24933814],[5],[2.81011278],[4],[3.60921511,3.60203389],[5,5],[2.63200371],[5],[3.29721911],[1],[4.54411195],[5],[3.15975447],[5],[3.59253284],[4],[3.32182904],[5],[3.93627702],[4],[3.46996803],[5],[2.94295464],[5],[4.6036581],[5],[2.75008978],[5],[2.47555583],[2],[3.10115698],[4],[4.19438985],[5],[3.43166472],[5],[3.55385205],[5],[4.35167496],[5],[3.76947119],[3],[3.2362063],[4],[3.13548676],[5],[4.26410036],[5],[3.00139496],[5],[3.68814838,3.49141498],[5,5],[3.54546502],[4],[4.05510586],[5],[3.6169518],[5],[3.82478102],[5],[2.7476608],[2],[4.64684804],[5],[2.80422713],[5],[4.16810369],[5],[3.09864162],[5],[3.67133165],[4],[4.06607688],[5],[3.43514954],[5],[3.64757901],[4],[3.0061326],[4],[3.06876305],[5],[3.28769375],[5],[3.0134919],[5],[3.72126911],[4],[3.54485936],[4],[3.69096332],[4],[4.57549293],[5],[3.22469348],[3],[2.4436206],[5],[2.88808098],[3],[4.30407624],[5],[3.35178565],[5],[3.35044253],[5],[3.22577824],[4],[2.8280253],[4],[2.96299152],[4],[4.26454317],[4],[2.11017897],[3],[4.12168799],[5],[3.5950424],[5],[4.07761291],[5],[3.38712409],[5],[2.97872068],[1],[3.46243775],[5],[4.67798632],[5],[3.58054761],[4],[3.01135078],[3],[4.19496834],[2],[3.49851412],[5],[2.9965886],[5],[3.85225853],[5],[4.47938306],[5],[2.498571],[1],[2.88488303],[4],[3.64117859],[3],[3.16439699],[5],[3.20502111],[3],[4.12847971],[5],[3.15754854],[5],[3.23119578],[4],[4.10105289],[5],[4.39499259],[5],[2.50684582],[2],[3.20218052],[5],[4.11462619],[5],[4.41745984],[5],[3.17648429],[3],[3.0128396],[5],[4.44506357],[4],[2.92436384],[4],[3.19277734],[5],[3.83979194],[5],[4.0827258],[4],[4.48655784],[5],[3.22541746],[5],[4.09222742],[4],[3.03974715],[5],[3.22638468],[5],[3.45455717],[4],[3.41312514,3.35242382],[5,5],[2.60023395,2.24117661],[5,5],[3.07470935],[4],[2.9081476],[5],[4.05324558],[5],[2.59352071],[4],[3.54883896],[5],[4.44812794],[5],[3.94744666],[5],[2.85885485],[5],[3.81354651],[5],[4.18166477,3.43016211],[5,5],[3.65786776],[5],[4.29448123],[3],[4.70018682,4.57594173,4.32217714,4.63763747],[5,5,5,5],[3.00028867],[5],[3.26749049],[5],[4.20323671],[5],[3.73799027],[5],[4.28172771],[5],[4.10537076],[5],[3.80823867],[5],[3.1410614],[4],[3.0288762],[5],[4.37314741],[4],[3.10828716],[3],[3.77653699],[5],[3.29575856],[4],[3.310807,4.50303344,2.08965663],[5,5,5],[4.0012081],[4],[3.26618614],[5],[3.96813803],[3],[3.09117084],[4],[3.14064388],[4],[3.32118719],[5],[3.27144877,4.29299448,3.98072232,4.50683388],[5,5,5,5],[3.00275991],[3],[1.58954652],[1],[3.01834061],[5],[3.97062451],[4],[2.99989338],[1],[3.3670261],[5],[3.92963224,3.51313588],[5,5],[3.20924756],[5],[3.98213814],[4],[4.42180872],[5],[3.88163695],[5],[3.02221649],[4],[3.62065058],[1],[3.59940813],[2],[3.05866609],[5],[3.23908704],[5],[3.07334453],[4],[4.62680996],[5],[1.92807723],[3],[4.19262771],[3],[3.0505297],[4],[3.11506496,3.20263078],[5,5],[3.72024973],[4],[4.25139873],[5],[2.7696278],[5],[3.15428276],[5],[3.7347617,2.77960213,3.0351326],[5,5,5],[2.72044842],[5],[2.60692083],[3],[4.18683247],[5],[3.33421572],[5],[4.59783702],[5],[2.72300287,3.37140118,3.11899343],[5,5,5],[2.49676612],[5],[4.05393581],[5],[3.17546667],[2],[3.18860037],[4],[3.15893455],[5],[4.80795261],[5],[3.37908255],[5],[2.72259195],[5],[3.03782],[5],[3.22523067,2.92548449],[5,5],[2.99634917],[2],[2.79243212],[2],[3.0838481],[5],[3.69396283],[2],[4.29041566],[4],[3.08314218],[5],[3.37878379],[5],[4.2206639],[5],[4.37583452,2.67969142],[5,5],[3.49264526],[4],[3.75938493],[5],[1.70545113],[2],[3.90047223,2.67928265],[4,4],[3.00088407],[5],[3.6809837],[4],[3.49541295],[4],[4.62753678],[5],[3.47889133],[5],[3.13982727],[3],[4.6100242,4.74929457,3.56602695],[5,5,5],[3.34918367],[5],[3.09952904],[5],[4.09089113],[3],[4.15139157],[5],[2.89375228],[4],[3.24979547],[4],[3.27951824],[4],[3.1961129],[4],[2.83280654],[4],[4.24317184],[4],[2.75360782],[4],[4.00700529],[5],[4.40164291],[5],[3.50138957],[5],[3.00124592],[5],[2.970049],[3],[2.9052336],[5],[2.78083186],[5],[4.24415636,3.81734674],[5,5],[3.00345285],[4],[4.10991869],[5],[3.11640599],[5],[2.49603691],[5],[3.88094425],[5],[3.03077405,4.24133089],[5,5],[3.83043667],[5],[3.25164945],[4],[4.53338643],[5],[3.43630177],[5],[3.2159416],[5],[3.75770194],[3],[3.34530969],[5],[3.51706525],[5],[2.83413244],[5],[4.34560041],[5],[3.83315459],[3],[3.26590516],[5],[3.36772838],[5],[2.84897969],[2],[3.3756645],[5],[3.37788575],[5],[2.99266363],[5],[3.64269459],[5],[3.52112622],[5],[4.06454998],[3],[2.88728438],[5],[2.74313233],[5],[2.81721024],[1],[3.9292586],[5],[3.50240896],[5],[3.46682562,3.31576061],[4,4],[3.00388767],[4],[3.16753565],[5],[3.97724994],[5],[3.06821626],[5],[3.76992252],[4],[2.93547392],[5],[3.94781807,2.72012623],[5,5],[3.30164999],[5],[3.20403115],[5],[3.60973918],[4],[3.15328363],[5],[4.13422938],[5],[3.41579872],[5],[3.49266614],[5],[2.92347479],[1],[4.55021744],[5],[3.09713134],[5],[2.79647034],[4],[3.77065488,3.14530964],[5,5],[3.50591865],[4],[4.56510269],[5],[4.04230497],[3],[3.78858417],[4],[3.85581569],[4],[4.3278758],[5],[3.08109291],[2],[3.16751932],[5],[4.20270641],[4],[3.23614025],[4],[3.610384],[4],[4.18472285],[4],[3.90292753],[5],[3.05355],[5],[2.99988878],[5],[3.00134951],[4],[3.35521911],[5],[4.7302849],[5],[3.35232834],[3],[3.40343639],[5],[3.84507608],[4],[3.43974037],[4],[3.372868,4.00680051],[5,5],[3.08183888],[2],[4.87162966,4.57651586],[5,5],[3.03237603],[5],[3.83546507],[4],[3.03905906],[5],[4.16278558],[4],[3.800976],[5],[3.28771788],[5],[3.35869174],[5],[2.74005594],[5],[3.59021232],[5],[3.28196806],[4],[3.0022315],[5],[4.16239785],[5],[4.72626202],[5],[3.49917733],[5],[3.85896769],[4],[2.90581124],[5],[2.71197963],[2],[3.97106206],[5],[3.33215378],[4],[4.36097603],[4],[4.12285218],[5],[4.03137237],[5],[3.03051632],[5],[2.99984299],[5],[3.17370107],[5],[4.24422312],[5],[3.00045776,3.00178369],[5,5],[3.35717718],[5],[4.63599755],[5],[3.40087926],[4],[2.33549903],[5],[3.30981942],[5],[3.3781096],[5],[2.71092784],[4],[4.25305107],[5],[4.6787459,3.64680681],[5,5],[2.99320465],[5],[2.49962287],[5],[2.59452121],[5],[3.57508968],[5],[3.1097023],[2],[2.93734948],[3],[4.18924854],[5],[4.56876186,4.10090565],[3,3],[4.7181836],[5],[3.24515011],[5],[3.53521495],[5],[4.69407002],[5],[3.18004474],[4],[2.93085562,3.03407133],[5,5],[3.55851312],[5],[3.57484476],[4],[3.2896092],[5],[4.69988749],[5],[3.1301938],[5],[3.76734746],[5],[2.78068044],[4],[3.46968845],[5],[3.61649081,3.94989238],[5,5],[3.28332907],[5],[4.27094434],[3],[2.6761396],[5],[3.43871561],[2],[4.75238193],[5],[3.38912877],[5],[2.84438037],[5],[3.2973132],[5],[2.81706565],[5],[3.26201308,2.7938772,3.66311167],[5,5,5],[3.59291244],[4],[3.78675481,4.12974874],[5,5],[3.8923002],[5],[4.58619988],[5],[3.26183242],[5],[3.62150315],[3],[3.33406456],[5],[3.72752496],[5],[3.77042209],[4],[3.53418552],[3],[2.91948372],[4],[3.36718823],[5],[4.08444846],[5],[2.7554935],[5],[4.30908587,4.43675633],[5,5],[4.18395746],[5],[3.04730024],[5],[4.43432542,4.60611534],[5,5],[3.25223554],[5],[3.00022451],[4],[4.86108601,4.66627326],[5,5],[3.43406705],[5],[4.74432508],[5],[2.87816151],[1],[3.88419008],[5],[3.07537054],[4],[4.25440885],[5],[3.35138524],[5],[4.32324328],[2],[2.63124176],[5],[4.37997204,2.77664591],[5,5],[3.6745153],[5],[3.25871382,2.82326329],[4,4],[2.98240248],[4],[3.37680385],[5],[3.03835531],[4],[2.90202303,2.99192763,2.94166375,3.22078491],[5,5,5,5],[4.19951291],[5],[2.64674882],[5],[2.84899937],[5],[3.30152047],[3],[2.498146],[5],[3.90083199,3.01866232],[4,4],[2.59140114],[3],[3.52411032],[5],[3.00978045],[3],[3.17895829],[5],[3.37435414],[5],[3.73688487],[5],[4.74421713],[5],[3.51704841],[2],[3.15768853,2.53485706],[5,5],[3.48483196],[5],[2.92108121],[4],[3.31211013],[2],[2.50974446],[4],[2.32963624],[5],[4.15479521],[5],[3.14351753],[5],[3.65302454],[5],[3.01839928],[5],[2.99942066],[2],[2.94774879],[5],[2.95287563],[5],[2.29113605],[5],[3.26089312],[5],[4.34296547],[5],[2.99640216],[5],[4.26480453],[4],[3.54441208],[4],[4.3887166],[5],[4.62616974,3.18093794],[5,5],[3.45814357],[5],[4.56885967],[5],[2.97240821],[4],[3.41132718],[5],[3.89959346],[3],[2.51415386],[5],[3.01334015],[3],[2.64664449],[5],[2.99993352],[4],[3.64450171],[5],[3.56352785],[3],[3.19862967],[5],[3.44654213],[5],[3.31038466],[4],[3.72267724],[5],[3.78273369],[5],[3.45660841],[5],[3.37039893],[4],[3.20808246],[5],[3.03193844],[4],[3.42339307],[5],[3.29789262],[4],[4.12127954],[3],[3.24602319],[5],[3.58439058],[4],[4.29563651],[5],[2.80493668],[5],[3.11095105,3.79836528],[4,4],[4.33028217],[4],[3.26843601],[2],[3.19970965],[5],[3.36342799],[4],[3.20923245],[4],[3.37330069],[5],[3.97142208,3.97462816],[5,5],[2.83811958],[5],[3.56351634],[4],[4.40507695],[5],[3.2015219],[3],[3.44913401],[4],[3.25456926],[5],[3.00075822],[5],[3.68568202],[5],[3.81127804],[4],[3.81120628],[4],[2.16871269],[1],[3.11418436],[4],[2.99826484],[5],[3.11767851],[3],[3.19649853],[5],[4.28755309],[1],[3.1437089],[5],[3.01757727],[5],[4.13311481],[5],[3.02507152],[4],[3.27826137],[5],[4.23027535],[5],[2.43136728],[4],[3.21894754],[5],[4.42386454],[5],[3.73670065],[4],[4.55185402],[5],[2.34705246],[1],[4.25254498],[5],[3.00267151,3.00008636],[5,5],[3.75683335],[5],[3.06439544],[5],[3.04175951],[5],[2.973761],[4],[3.60095501],[5],[3.49046791],[5],[3.44497811],[2],[3.30377651],[3],[3.75646135],[3],[3.02407293,3.05931416],[5,5],[3.33366014],[2],[4.29613181],[5],[3.88560547],[5],[3.16549747],[3],[3.01450474],[5],[3.10322439],[3],[3.03826223],[3],[4.84091197,4.78164055,4.81745866,4.8963123],[5,5,5,5],[3.59172204],[5],[3.32913008],[5],[3.45005315],[4],[2.83029085],[5],[3.55250482],[5],[3.76365304],[4],[2.89044491],[5],[4.84901837],[5],[3.1200663],[5],[4.01745807],[5],[3.99407463],[5],[2.99757864],[3],[2.92366296],[4],[3.48071954],[1],[3.57032986,4.59032873],[5,5],[2.97016011],[5],[3.35394167],[5],[3.02666652],[5],[2.35368894],[3],[2.83623169],[5],[3.44609056],[5],[2.30783756],[5],[3.4184928],[4],[3.4160253],[4],[3.53205234],[5],[4.36771957],[5],[2.82468662],[4],[3.69642453],[5],[3.3555104],[5],[3.70660213],[5],[3.62270549],[5],[3.05770311],[5],[2.99424655],[4],[2.99588838],[5],[3.10942542],[5],[3.25722726],[4],[3.27966927],[5],[2.99570865],[5],[2.1810621],[4],[3.64873173],[5],[4.10298927],[4],[2.9545302],[4],[4.51013392],[5],[4.62899604],[5],[3.20515032],[2],[3.70646191],[5],[3.33415514],[4],[3.45945221],[5],[3.99844147],[4],[3.11005732],[4],[2.65770035],[5],[2.0335919],[1],[2.87568004],[5],[2.77566523],[4],[4.19423881],[3],[3.86222812],[5],[4.59754821],[5],[2.61942991],[5],[2.9947577],[3],[3.52436341],[5],[3.62458245],[4],[3.19448061],[5],[3.37347166],[4],[2.98426562],[5],[3.03131071],[5],[3.21481459],[4],[3.35521408],[4],[2.60328397],[4],[3.00632201],[4],[3.68103504,3.66425224],[5,5],[3.79301691],[5],[3.77033218],[5],[3.1131605],[5],[3.73452229],[3],[3.98787242],[4],[3.25238986],[5],[3.18993844],[5],[4.05667162],[5],[2.80159618],[5],[2.73044642],[4],[3.45407261],[5],[2.9685553],[5],[4.648607],[5],[3.31481695],[5],[3.19857358],[5],[2.96644041],[5],[4.16310028],[5],[4.1996848],[5],[2.69193359],[3],[4.80869493,3.61334409],[5,5],[3.91912317],[5],[3.24438509],[5],[3.58656593],[5],[4.14860082],[5],[3.10010203],[5],[3.34389082],[5],[3.63244007],[5],[4.17252482],[5],[3.03169141],[5],[2.89837607],[5],[2.85329857],[5],[4.60495316],[5],[3.00522222],[3],[3.73957383],[5],[3.93011826],[4],[2.99397669,3.10825357],[5,5],[4.28642736],[5],[3.76418467],[3],[3.76304563],[5],[3.39559163],[5],[2.92694116],[5],[3.45513268],[5],[3.37699423],[5],[3.79899921],[5],[2.76247013],[3],[3.08817795],[3],[4.21603046],[5],[3.05201557],[5],[3.69884805],[5],[2.64328822],[4],[3.5520653],[4],[3.16792537],[5],[4.77788304],[5],[2.39012586],[5],[3.33847345],[4],[3.29079788],[4],[3.00025415],[5],[2.99615622],[3],[4.47908885],[5],[2.74133797],[5],[3.89873534],[3],[3.46794001],[5],[3.60379188],[5],[4.11702872],[5],[3.00131566],[5],[3.3064704],[5],[3.08335704],[5],[4.71359888],[5],[2.95649975],[5],[3.46788268],[3],[3.73200511],[5],[3.02991343],[5],[3.70956272],[5],[4.01573365],[4],[2.90669235],[4],[3.69031883],[4],[4.37948712],[5],[2.57897607],[5],[3.81616488],[5],[3.99080952],[5],[4.30484464],[3],[3.25025274],[5],[3.34061774],[5],[4.1016867],[5],[3.3940738],[5],[3.01428422],[3],[4.09367465],[4],[3.49007019],[4],[2.7917838],[5],[3.1318055],[4],[2.73484343],[5],[4.34885748],[4],[3.11601369],[5],[3.91337162],[5],[2.94285146],[5],[4.16627118],[5],[2.86879982,3.02188858],[4,4],[4.23136742,2.78698544],[5,5],[3.06134254],[5],[3.38249587],[4],[2.7120804],[3],[4.22609848],[5],[4.15492117],[5],[2.70920654],[4],[3.6210336],[5],[4.36058314],[5],[3.16727027],[5],[3.934422],[5],[3.31998598],[5],[3.17367728],[5],[3.62711169],[4],[3.78966842],[3],[4.38709433],[5],[3.13872787],[5],[4.21058882],[5],[3.56411469],[3],[3.05826812],[5],[3.96131965],[1],[3.96539147],[4],[3.07143395],[5],[2.88735477],[3],[4.88500573],[5],[4.41610434],[4],[3.46658389],[5],[3.37892632],[3],[3.62862332],[5],[3.94859277],[4],[3.08674347],[4],[2.52712048],[5],[3.41094163],[5],[3.93629803],[5],[3.64192498,3.59179439],[5,5],[3.00251817],[3],[3.37219752],[5],[2.40035499],[5],[4.47732675],[5],[3.32439844],[5],[3.69288422],[5],[3.76026838],[5],[3.42844084],[3],[3.7598127],[5],[4.47660823],[4],[2.90318168],[5],[4.59599663],[5],[3.1179589],[4],[3.17984727],[3],[3.13385722],[5],[3.22925631],[4],[2.53652008],[5],[3.26670678],[4],[2.7754407],[4],[3.69243738],[4],[3.24292297],[5],[3.0649967],[3],[3.10386603],[1],[4.35086395],[5],[4.68392245],[5],[3.28295501],[5],[2.5491134],[3],[3.16824307],[5],[3.7165643],[4],[3.83043362],[5],[3.36462125],[5],[2.74999024],[5],[3.2988025],[4],[2.39343436,3.10951028],[5,5],[2.9216825],[2],[3.20803536],[5],[2.96081886],[5],[4.16366449],[5],[4.79680783],[5],[3.6710994],[5],[4.1828145],[5],[2.82635532],[5],[3.45889509],[5],[4.04010418],[5],[4.63239062],[5],[3.99834462],[5],[3.00162923,3.00339979],[5,5],[3.07696809],[1],[3.8175072],[4],[3.26524104],[4],[3.22100973],[3],[2.2783779],[1],[4.34434682],[5],[3.00002632],[5],[3.4720202],[3],[3.38807665],[5],[3.61253351],[5],[3.37156846,3.81246212],[5,5],[3.06492053],[5],[4.43542944],[5],[2.9970056],[4],[2.90869523],[5],[4.16586624],[5],[2.83348212],[5],[3.43110194],[5],[2.72066679],[5],[3.05184848],[5],[2.7888731,2.85776585],[5,5],[4.74626069],[5],[4.10659076],[5],[3.38488938],[5],[3.60266819],[3],[3.72264846],[5],[4.46830279],[5],[3.08548702],[4],[3.11733532,3.1662839],[4,4],[4.31449073],[5],[3.12588045],[5],[3.57768919],[4],[4.15651843],[5],[2.84405688],[5],[3.54512909],[4],[2.99686411,3.1173789],[5,5],[3.81413441],[5],[2.87278575],[5],[3.81431785],[5],[2.91682674],[4],[2.24813925],[4],[4.14641843],[5],[4.26922578],[5],[2.95880589],[5],[4.25928793],[5],[4.10797917],[4],[3.51797105],[4],[3.17303177],[5],[3.13847623],[3],[3.14517797],[5],[3.33667143],[5],[2.92375459],[4],[2.64058435],[5],[3.69010643],[4],[4.27013967],[5],[3.49246317],[5],[3.00369167],[3],[4.1416656],[5],[1.89563148],[5],[3.53283622],[5],[3.45471742],[4],[2.9375559],[4],[3.11578988],[2],[2.68670103],[4],[2.73821205],[1],[3.18487092],[3],[3.64469952],[5],[2.87823183],[5],[1.79157808],[1],[1.51384083],[1],[2.5233742],[5],[2.99805274],[3],[3.24114436],[4],[3.11255048],[2],[3.17506258],[4],[2.88587984],[5],[2.82181294,2.17064406],[5,5],[3.78272312],[5],[3.32459618],[5],[3.21250308],[5],[3.09432722],[5],[2.97490188,3.3310155],[5,5],[3.57458576],[5],[3.50653526],[4],[3.17138028],[5],[3.07344904],[5],[4.000423],[4],[4.25925674],[4],[3.5006087],[3],[3.05396159],[5],[3.23177647,3.07167131],[5,5],[3.67069044],[4],[2.82681516],[5],[2.99597241],[5],[3.11194114],[5],[3.3219019],[3],[4.50761613],[5],[4.43643228],[5],[3.43620071],[2],[4.4482597],[5],[3.92709281],[5],[3.24998316],[5],[2.95146769],[5],[3.05733665,3.53655463],[5,5],[3.59748478],[4],[2.95982071],[5],[3.00598813],[4],[3.14595842],[5],[3.81382447],[5],[3.35571361],[4],[2.89297469],[5],[3.21302519],[5],[3.27364247],[3],[3.37133356],[4],[3.57283136],[1],[3.05679833],[5],[2.79029251],[5],[4.49856463],[5],[2.93454777],[5],[2.9194987],[5],[3.23106927],[5],[2.86992819],[4],[3.28490861],[5],[3.38102038],[5],[3.7605002],[4],[3.01249914],[5],[4.06630723],[5],[2.92273199],[3],[3.1664255,4.2001172],[5,5],[3.27714936],[4],[3.68450277],[5],[3.52092654],[5],[3.58370073],[5],[4.29400111],[4],[2.80220186],[5],[3.13171641],[5],[3.38178713],[5],[4.17008181],[5],[3.09239392],[4],[2.98277061],[5],[3.37962071],[5],[2.50774953],[5],[3.00143884],[3],[4.33656818],[5],[3.23579141,3.82620915,3.47163122],[5,5,5],[3.17326756],[5],[3.21337685],[5],[3.01190958],[5],[2.7415148],[5],[3.89656856],[4],[2.52974396],[5],[3.58891796,3.65547097],[4,4],[4.01719431],[5],[3.33855208,3.99716621],[5,5],[3.21371507],[5],[2.94663442],[5],[2.70725216],[5],[3.4724633],[5],[3.24619577],[3],[2.60472629],[5],[3.40407514],[5],[3.74007248,4.32916024],[5,5],[3.53275203],[3],[2.98749346],[5],[2.45745843],[5],[2.99503681],[4],[2.29839513],[4],[2.96263179],[5],[3.70987042],[4],[2.52272016,2.95935031],[5,5],[2.89738566],[5],[3.62831061],[5],[3.20249472],[3],[3.88509715],[4],[3.2060458],[5],[3.00022679,3.0050144],[5,5],[3.61408213],[5],[4.13179346],[5],[4.25311053,4.21474136],[5,5],[2.69863487],[2],[3.95969192],[5],[2.9986541],[3],[3.41284155],[4],[3.09007325],[5],[3.46382535],[3],[4.05180311],[5],[4.26269385],[5],[2.88889723,2.81018101],[5,5],[3.44999688],[3],[3.49219677],[5],[2.69675835],[4],[4.65470222],[5],[3.98976109],[2],[3.08313887],[5],[3.41526753],[4],[2.74534199],[5],[3.35639458],[4],[3.09051686],[3],[2.99835658],[5],[3.32074921],[5],[3.00003524,3.00159254],[4,4],[2.58990401],[5],[2.3941502],[2],[3.50800805],[4],[3.0603635],[4],[3.1184878,3.19088048],[4,4],[4.53388356],[5],[3.26746709],[4],[4.09657753],[4],[4.79888935],[5],[3.25483009],[5],[3.54730125],[5],[3.34087845],[3],[3.52989862],[5],[4.14700012],[5],[2.87265342],[4],[3.20096258],[5],[3.1240901],[5],[2.79060858],[5],[4.41325262],[5],[2.72495151],[5],[3.15818327],[4],[3.45534134],[3],[2.78656804],[5],[3.4081848],[5],[4.46244025],[5],[4.38624132],[5],[3.50220788,2.8076966],[5,5],[3.64842213],[5],[3.5888286],[5],[4.79182964],[3],[3.20163452],[5],[3.25611445],[5],[3.28883953],[3],[3.19076425],[5],[2.79405232],[5],[2.84869494],[5],[3.97172127],[5],[2.94112316],[5],[3.16903271],[5],[2.91670192],[4],[2.81974552],[3],[2.59498349],[1],[3.77447249],[5],[2.88263471],[4],[3.67428898],[5],[3.10207744,3.20486474],[5,5],[3.7439617,2.6973962],[5,5],[2.65691037,4.44474542],[5,5],[3.4452793],[3],[3.35638456],[5],[2.6056253],[5],[2.58890488],[2],[4.14628543,3.08489966],[4,4],[3.32870087],[5],[3.95910235],[4],[2.98068772,3.81929995],[4,4],[2.8558313],[5],[3.09554214],[5],[3.14005276],[3],[3.1463821],[4],[3.01859345],[4],[3.16293839],[5],[3.51179284],[4],[3.92494673],[4],[3.31373868],[5],[3.33878263,3.5958254],[5,5],[4.35093931],[3],[1.99501501],[2],[3.03056724],[5],[2.33038112],[5],[3.08680912],[4],[3.5111671,3.46890442,4.13716727],[5,5,5],[3.60588332],[5],[3.44273916],[5],[3.25370257],[5],[2.81387841],[5],[3.0347509],[5],[4.27577741],[5],[3.48892275,4.37525395],[5,5],[4.01049145],[5],[2.99773372],[5],[3.2365133,3.31037784,2.73678834],[5,5,5],[2.8670236],[5],[3.72482156],[5],[3.14006551],[5],[3.07722986],[5],[4.09921936],[5],[2.94765241],[3],[3.15328433],[4],[3.72539058],[5],[3.86118672],[5],[3.09454637],[5],[2.88032491],[5],[2.86672394],[5],[4.50468045],[5],[2.99491944],[1],[4.31764262],[5],[3.00057771],[2],[3.14932882],[5],[3.14441208],[5],[4.12185161],[5],[4.40229775],[5],[3.23380303],[5],[2.93631989],[3],[3.0212146],[4],[4.43181164],[5],[2.73047486],[4],[3.2533535],[5],[4.34526798],[5],[2.37492115],[4],[4.08330644],[5],[3.61730749],[5],[2.77277335],[5],[3.20120308],[5],[3.3594491],[5],[3.21966751],[5],[4.86187521],[5],[3.94863088],[5],[2.84216676],[5],[3.17427942],[5],[4.67648887],[5],[3.15969895],[5],[2.9653205,3.50193886],[4,4],[3.36793024],[5],[3.18012186],[3],[3.27427288],[3],[3.19494344],[4],[4.40782693],[4],[4.33870991],[2],[4.61396403],[5],[3.75537],[5],[4.0612074],[5],[3.0449543],[5],[4.22213787],[5],[2.03778534],[5],[3.90065222],[5],[3.92089343],[5],[4.40798601],[5],[4.26557716],[4],[3.40407949],[5],[3.12597747],[5],[4.50209796],[4],[4.6410107],[5],[4.63432987],[5],[3.19518057],[4],[3.27712473],[2],[2.68987855],[5],[3.41480149],[5],[3.35738474],[5],[3.85632297],[5],[4.30310521],[5],[4.53469396],[1],[4.08994465],[5],[3.18401159],[4],[4.3370141],[5],[3.1685642],[5],[3.04521043],[5],[2.93632006],[1],[4.08395312],[4],[2.99782632],[1],[2.93025948],[1],[3.01135103],[2],[3.04597108],[2],[3.26787688],[3],[2.33390793],[4],[3.31330905],[5],[3.19893682],[5],[4.20932203],[5],[4.00312666],[2],[3.05213101],[5],[3.1060817,2.84174766],[5,5],[2.94774282],[3],[3.07544803,3.1241636],[5,5],[2.90075804],[5],[3.27623015],[5],[3.42693892],[5],[2.94933737],[5],[3.26357298],[4],[3.14249232],[5],[3.25651305],[2],[3.20569475],[5],[3.2022229],[5],[3.23490634],[3],[3.00220723],[5],[3.30560157],[3],[3.04647966],[5],[2.68737634],[5],[4.1025339],[5],[3.31982223],[5],[2.90651804],[5],[2.8184557],[3],[3.13409395],[5],[3.5238194],[5],[3.11100635],[5],[3.21101462],[5],[3.53727622],[5],[3.12229024],[1],[3.42357663],[5],[3.09968857],[5],[3.20639315],[5],[2.32680249],[3],[3.85044426],[5],[3.0436599],[2],[2.85865714],[4],[3.11722386],[5],[4.01651381],[5],[3.13933663],[4],[4.30827807],[5],[3.0056789],[4],[3.56185646],[5],[2.97111942],[1],[3.27146371],[5],[3.29191089],[3],[3.21485781],[3],[3.22355607],[4],[2.98030481],[5],[3.87884609,3.86683077],[4,4],[2.81645657],[5],[3.02170731],[4],[3.33817991],[5],[3.83360047],[3],[2.99959604],[5],[2.88173764],[4],[3.50261313],[5],[3.78675157],[5],[4.35441649],[5],[3.85571251],[5],[3.93271358,3.89067537],[4,4],[3.54743263],[5],[3.14020106],[2],[4.25885644],[5],[4.15161232],[4],[3.11556713],[3],[3.66621365],[5],[2.32023244],[1],[3.4857472],[3],[3.05970467],[4],[3.00381488],[4],[3.05962881],[5],[3.07595123],[4],[1.88568589],[5],[2.70287316],[4],[3.04669678],[4],[2.72228054],[2],[3.27461639],[5],[3.09438977],[4],[4.09380932],[5],[3.10261935],[3],[2.87988194],[4],[2.4587084],[4],[4.51844381],[5],[3.64333703],[2],[3.70827142],[4],[4.08047127],[5],[3.23534371,3.21242413],[5,5],[3.002334],[3],[3.25546088],[3],[2.99952211],[2],[3.5921302],[5],[3.97199403],[5],[3.37524656],[5],[2.65028359],[5],[3.32909352],[5],[3.09956663],[5],[2.97732778],[5],[3.16824477],[4],[3.92787861,3.10928446],[5,5],[2.84158019],[5],[3.52939796],[5],[3.10725023],[5],[3.10158661,3.34293805],[5,5],[2.74940141],[5],[3.62676265],[5],[3.21225763],[5],[3.42197709],[4],[2.60930415],[5],[3.94392745],[5],[2.94196143],[5],[2.79549833],[5],[3.55663375],[5],[3.13579204],[5],[3.4784857],[5],[4.44319976],[3],[3.86800478],[5],[3.33856765],[5],[2.67727788],[5],[3.16597418],[5],[2.47006436,2.61583902,4.11817569,3.97664069],[5,5,5,5],[3.18729472],[5],[2.62711751],[5],[2.86245401,2.89059152],[5,5],[3.05552993],[5],[4.00728877],[5],[3.45693133,2.88593202],[5,5],[3.41771201,3.13430397],[5,5],[2.90995146],[5],[3.34016804],[5],[4.38701619],[5],[3.11279616,3.43757379],[4,4],[3.10574497],[4],[3.87968981],[2],[3.19987841],[5],[3.13144672],[5],[2.95383189],[5],[3.2332819,3.8199501],[5,5],[3.82547655],[4],[3.56539197],[5],[2.83788143],[4],[3.13456265],[5],[3.19319678,3.100132],[4,4],[1.82643107],[4],[2.80281105],[5],[3.51526537],[3],[3.13745129],[5],[2.79277405],[5],[4.12451513],[5],[4.42180948],[5],[3.41583426],[5],[3.04664446],[4],[2.95902895],[4],[3.13358929],[5],[3.59467119],[5],[4.52793327],[5],[2.73502392],[5],[3.53550696],[3],[1.43679943],[1],[3.59508992],[4],[3.74508979],[4],[3.23716365,3.21579613],[5,5],[3.14776679],[5],[2.93481986],[5],[3.15264329],[3],[3.23766051],[4],[4.15909762,3.9926658],[5,5],[3.15129768],[5],[3.40288251],[1],[4.13453273],[5],[2.92268431],[4],[2.83276278,3.1888955],[5,5],[2.49739556],[5],[3.67825682,3.91632167],[5,5],[2.74630836],[5],[3.07171015],[5],[2.3087212,3.49705226],[5,5],[3.37779322],[4],[4.38995069],[5],[2.65507503],[3],[3.07242657],[4],[3.32808201],[5],[3.28913935],[2],[3.40458802],[2],[3.78500732],[3],[4.73567204],[5],[4.06876204],[5],[2.95643343],[3],[2.93153259],[2],[3.88283445,3.37538968],[5,5],[2.99712813],[5],[3.07629316],[5],[2.8100112],[5],[3.02337658],[1],[2.87102644],[4],[3.13098065],[5],[2.94239336],[5],[3.21981883],[3],[3.05833427],[3],[3.14502804],[4],[3.50166614],[5],[3.59856364],[5],[4.20239439],[5],[2.84410827],[4],[2.57802035],[4],[3.00249616],[5],[4.25276832],[3],[4.58927917],[5],[3.3094831],[5],[2.2345998],[5],[4.63955792],[4],[3.9822368],[4],[3.41813304],[1],[3.47410445],[4],[3.67413372],[5],[3.57759245],[5],[3.19101253],[5],[3.77287421],[5],[3.76677634,3.22040494],[5,5],[3.55941762],[4],[3.37546888],[5],[3.29176812],[5],[2.88613545],[5],[2.81209286],[5],[3.46531714],[4],[3.58111164],[5],[3.38208288],[1],[2.99963466],[5],[1.72497742],[5],[3.00393778],[3],[3.49077339,2.973782],[5,5],[4.39488509],[5],[2.99900548],[5],[2.72588955],[5],[3.43180824],[5],[2.7347019],[5],[3.21820839],[5],[3.21544485,2.76812652],[5,5],[3.08070571],[5],[2.54337515],[5],[3.01635658],[5],[3.16053625],[5],[2.75605236,3.06260916],[5,5],[3.58407306],[5],[3.81072524],[3],[2.90755949],[1],[2.9982946],[3],[3.31864107,2.78574314],[5,5],[4.80746455],[5],[2.89724857],[5],[3.38197613],[4],[4.67245232,4.59201457,4.55073685],[5,5,5],[3.221017],[2],[3.7596171],[5],[4.11148981],[3],[3.1856753],[5],[3.44537021],[3],[3.4665462],[2],[3.32840454],[5],[3.05475919],[5],[3.24687766],[5],[3.17081498],[5],[2.54268989],[4],[2.63767996],[2],[4.47229941],[5],[2.32437414],[5],[2.9879509],[4],[3.14170361],[5],[3.21241847],[4],[3.55061787],[3],[2.6954413],[3],[3.04164999],[3],[4.26635491],[5],[2.93077588],[5],[2.42245999],[5],[3.71283514],[3],[4.04126285],[4],[2.76370126,2.7478956],[5,5],[2.97687587],[4],[3.81398633],[4],[4.06329558],[4],[4.76102042],[5],[4.63018712],[4],[3.33688986],[5],[3.7098034],[5],[3.21234921],[2],[4.63574216,4.51082226,4.56783563],[5,5,5],[2.76137809],[5],[3.14704708],[5],[3.4191813,2.20004972,4.72502112],[5,5,5],[2.94096676],[3],[2.69163583],[1],[3.49714101],[5],[2.93670385],[4],[3.35118933],[3],[3.93403464],[5],[3.1057971],[5],[2.96376511],[5],[2.84396771],[5],[3.23101228],[5],[3.26207837],[5],[2.82625268],[2],[3.68564132,3.86191133],[4,4],[4.03244363],[5],[2.63831254,2.87612343],[5,5],[3.70174735,4.03018614],[5,5],[3.91292733],[5],[2.90565431],[4],[2.98601954],[5],[2.73261403],[4],[2.87572759],[4],[3.38969171],[5],[3.11490047],[5],[3.05252118],[4],[3.53428285,3.55082636],[5,5],[3.04020442,3.16816088],[5,5],[3.50959839],[3],[2.41657239],[5],[2.07748574],[1],[2.90882242],[4],[4.0965053],[1],[2.92268711,3.01166243],[5,5],[2.99941981],[5],[2.93535244],[5],[3.25991214],[5],[3.1116302],[5],[3.09604643],[5],[4.31644007],[5],[2.63206397],[5],[1.71872231],[1],[3.20969642],[5],[3.51129056],[5],[4.14267323],[5],[3.35784675],[4],[3.29516467],[3],[3.20160967],[5],[2.93431555],[4],[2.99846517],[5],[3.00588281],[5],[3.22642439],[4],[3.58079244],[5],[4.14340902],[5],[4.38462826],[5],[3.09702086],[4],[3.67817492],[4],[3.00204886],[5],[3.28830951],[5],[4.30313567],[4],[2.54750203],[4],[2.49249937],[5],[3.28378157],[5],[2.6644769],[5],[3.15027317],[5],[3.23750899],[5],[3.44280403,3.25091272],[5,5],[3.00017368],[3],[3.50237837],[1],[3.90917721],[5],[3.22499817],[5],[4.63652804],[5],[2.966571],[4],[2.75510527],[4],[3.56233817],[5],[3.38986048],[5],[2.8140384,2.9927392],[5,5],[4.40933925],[5],[3.24178029],[5],[3.6339091],[5],[3.45839394],[5],[3.2398503],[5],[3.53628459],[5],[3.04867934],[5],[3.47760991],[5],[3.48329867],[5],[3.51750737],[5],[4.35521483],[5],[3.29775574],[5],[3.15813199],[5],[2.85524099],[5],[3.32104882,3.14744173],[5,5],[3.24587527],[4],[3.26823377],[5],[3.02161879],[5],[3.40215649],[2],[4.14026467],[5],[3.18322617],[5],[3.18497246],[4],[3.92309579],[5],[4.17419659],[5],[2.98978512],[5],[4.21904783],[5],[2.72257357],[5],[3.00035815],[3],[3.64205159],[5],[2.69152438],[5],[3.15520974],[5],[3.78738815],[5],[3.60197849],[1],[2.99772377,3.00068365],[3,3],[3.67016404],[5],[3.21677239],[5],[2.9226467],[5],[3.30479505],[5],[3.14015528],[5],[3.1146666],[5],[2.63790868],[5],[2.75463456],[5],[3.18882199],[5],[3.16015639],[5],[3.29863037],[5],[3.02398126],[2],[2.65691701],[5],[3.46284506],[5],[3.92961117],[5],[3.03580543],[4],[4.13905764],[5],[2.91492019],[5],[2.98948854,2.86230114],[5,5],[3.5757722],[5],[4.11762525],[5],[2.6886613],[5],[3.78278735],[4],[4.36630653],[5],[3.34099656],[3],[4.33773656,4.06737639],[5,5],[3.27911661],[5],[2.32660631],[5],[3.2103968],[5],[3.38110024],[1],[3.94711938],[1],[3.19166467],[5],[3.55346826,3.66929345],[5,5],[3.19869751,3.51007648],[5,5],[2.9028799,2.90908047],[5,5],[4.32122676],[4],[3.22959163],[5],[3.93511972],[5],[3.32674361],[5],[3.64562471],[5],[3.45584461,3.62130029],[5,5],[3.17462078],[5],[3.10982756],[5],[3.27580152],[5],[3.84255927],[4],[2.7011988,2.54018456],[5,5],[3.08204524],[5],[3.30863829],[5],[3.86024089],[4],[3.34020891],[5],[3.15834799],[4],[3.2381252],[5],[2.86005297],[5],[3.39759846],[5],[4.15873219],[4],[3.22684103,3.09579352],[5,5],[3.29028272],[5],[3.94107321],[1],[3.33723506],[1],[3.00211551],[5],[3.41195537],[5],[3.35187403],[5],[3.00943854],[5],[2.67731165],[5],[2.56942426],[4],[3.21797756],[4],[3.22738684],[4],[3.30167474],[4],[3.51427644],[5],[3.12814957,3.01077016],[3,3],[3.41154076],[5],[2.94676563],[4],[3.18684506],[5],[2.87120822],[5],[3.16382707],[5],[3.64866121,3.67309025],[5,5],[3.00311921],[5],[4.0878651],[5],[3.02518594],[5],[2.83682617],[5],[3.06617613],[2],[3.41729644],[5],[2.84038884],[5],[3.0750204],[4],[2.91079419],[4],[3.26130572],[4],[2.99901532],[4],[3.51604805],[5],[2.85648212],[4],[3.88795528,2.95931783],[5,5],[4.12540867],[5],[3.27346597],[5],[3.54944486],[5],[2.67976104],[5],[2.95206378],[3],[3.04793826,3.04797955],[5,5],[3.15269943],[4],[3.05246168],[4],[3.17030106],[5],[2.45054641],[5],[3.51836701],[1],[2.99740115],[1],[3.68862486],[2],[2.99962793],[4],[3.30316495],[5],[3.01025383],[3],[3.01036626,3.54019415],[5,5],[2.99882049],[3],[3.03325622,3.02172347],[5,5],[3.41419083],[2],[3.27867432],[5]]
# truncated_gt_ratings=data[1::2]
# truncated_pd_ratings=data[::2]
# print (truncated_pd_ratings, truncated_gt_ratings)
# def sum_elements_in_arrays(arrays):
#     return sum(sum(sum(items) for items in array) for array in arrays)

# arr= ((sum(items) for items in array) for array in arrays)

# print(sum_elements_in_arrays([truncated_gt_ratings])/len(truncated_gt_ratings))


In [25]:
# mean_gt_ratings = np.mean(truncated_gt_ratings)
# tss = np.sum((truncated_gt_ratings - mean_gt_ratings) ** 2)
# rss = np.sum((truncated_gt_ratings - truncated_pd_ratings) ** 2)        

## Further discussion

*From image to graph*. If we were given some visual auxiliary data (e.g., item images), then we can take the same approach and instantiate our `GraphModality` using visual features. The only difference is that we would rely on the `ImageModality` to prepare our visual features.

*Cross-modality comparisons.* Note that beyond addressing the lack of a specific type of auxiliary data, another benefit of the kind of transformations addressed in this tutorial, is to enable quick and convenient cross-modality comparisons. In our example, if we have access to both visual and textual item content, we can quickly investigate which auxiliary information helps most, given MCF as our recommender model.